# 리뷰 - 리뷰 평점과 구매의 상관관계

In [190]:
import pandas as pd
import pymysql
import sqlalchemy
from urllib import parse

user = 'hyun'
password = 'hyun'
host='172.20.224.1'
port = 3306
database = 'KPI'
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

In [2]:
customers = pd.read_sql_query("SELECT * FROM customers", con=engine)
locations = pd.read_sql_query("SELECT * FROM locations", con=engine)
order_items = pd.read_sql_query("SELECT * FROM order_items", con=engine)
orders = pd.read_sql_query("SELECT * FROM orders", con=engine)
payments = pd.read_sql_query("SELECT * FROM payments", con=engine)
products = pd.read_sql_query("SELECT * FROM products", con=engine)
reviews = pd.read_sql_query("SELECT * FROM reviews", con=engine)
sellers = pd.read_sql_query("SELECT * FROM sellers", con=engine)

## 내가 막혔던 부분(리뷰 평점에서부터 막힘)
- reviews 에서 Review_id / Order_id가 일치하는 여러 개의 리뷰가 있었음
- order_items 에서 Order_id가 일치하는 여러 개의 행이 있었음
- order_items 에서 Order_id가 일치하는 여러 개의 제품 구매 이력인 게 reviews에서 물건 구매 종류 및 갯수와 맞지 않는 리뷰를 작성함

### reviews 에서 Review_id / Order_id가 일치하는 여러 개의 리뷰 해결 과정

In [3]:
reviews

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
0,REVIEW_00000,ORDER_01674,4,2019-01-18 00:00:00,2019-01-18 21:46:59
1,REVIEW_00001,ORDER_80140,5,2019-03-10 00:00:00,2019-03-11 03:05:13
2,REVIEW_00002,ORDER_69816,5,2019-02-17 00:00:00,2019-02-18 14:36:24
3,REVIEW_00003,ORDER_24398,5,2018-04-21 00:00:00,2018-04-21 22:02:06
4,REVIEW_00004,ORDER_70366,5,2019-03-01 00:00:00,2019-03-02 10:26:53
...,...,...,...,...,...
87868,REVIEW_87298,ORDER_58840,5,2019-07-07 00:00:00,2019-07-14 17:18:30
87869,REVIEW_87299,ORDER_75162,5,2018-12-09 00:00:00,2018-12-11 20:06:42
87870,REVIEW_87300,ORDER_08690,5,2019-03-22 00:00:00,2019-03-23 09:10:43
87871,REVIEW_87301,ORDER_25681,4,2019-07-01 00:00:00,2019-07-02 12:59:13


In [6]:
# 오더 아이디가 겹치는 것 확인
reviews.duplicated(['Order_id']).value_counts()

False    87349
True       524
Name: count, dtype: int64

In [8]:
reviews[reviews.duplicated(['Order_id'])]

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
1001,REVIEW_01002,ORDER_08418,5,2018-11-07 00:00:00,2018-11-10 20:07:48
2767,REVIEW_02773,ORDER_68723,5,2019-01-18 00:00:00,2019-01-18 17:36:45
7216,REVIEW_07224,ORDER_85111,5,2019-03-23 00:00:00,2019-03-24 00:23:06
8059,REVIEW_08068,ORDER_04544,5,2018-10-21 00:00:00,2018-10-22 00:47:25
8712,REVIEW_08722,ORDER_14054,3,2018-09-14 00:00:00,2018-09-17 11:14:08
...,...,...,...,...,...
87473,REVIEW_86910,ORDER_40011,5,2019-04-03 00:00:00,2019-04-10 14:38:27
87667,REVIEW_53847,ORDER_39488,5,2019-02-17 00:00:00,2019-02-19 19:29:19
87768,REVIEW_06134,ORDER_09043,4,2018-07-03 00:00:00,2018-07-05 19:06:59
87817,REVIEW_37828,ORDER_44549,5,2019-03-03 00:00:00,2019-03-04 22:56:47


In [15]:
reviews.loc[reviews['Order_id']=='ORDER_08418']
# Order_id가 같은 거 하나를 불렀을 때, 작성 시기와 리뷰 아이디가 다름

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
922,REVIEW_00923,ORDER_08418,5,2018-10-31 00:00:00,2018-11-04 21:12:29
1001,REVIEW_01002,ORDER_08418,5,2018-11-07 00:00:00,2018-11-10 20:07:48


In [24]:
order_items.loc[order_items['Order_id']=='ORDER_08418']
# 해당 Order_id 를 조회한 결과, 한 오더 내에서 한 제품을 1개 구매함
# 일주일 후기처럼 처음 후기를 작성한 후에 일정 시간이 지나면 재작성할 수 있는 시스템이 아닌지 추측

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
31684,ORDER_08418,1,PRODUCT_06296,SELLER_1059,196.99,56.66


In [23]:
# Order_id가 같은데 리뷰가 여러 개인 경우에는 평균으로 합쳐 하나로 보는 게 맞다고 봄
# 예시는 리뷰 점수가 같으나, 리뷰 점수를 동일하게 주지 않을 수 있으므로
# 한 주문에 대한 동일인의 리뷰이기 때문에 리뷰 점수의 평균 값을 구하여 1개로 합치는 것이 옳다고 판단
# Order_id 가 일치하지 않으나, 구매자는 같은 경우도 있을 수 있지 않은가?라는 생각이 들긴 했으나
# 다른 Order_id 로 구매하는 기간에 제품이 변경(리뉴얼 등)이 있을 수 있으므로
# 위의 경우에는(구매자는 같으나 Order_id 가 다른 경우) 하나의 리뷰로 합치지 않고 각각의 리뷰로 볼 것

In [26]:
# 근데 이제 문제를 새로 찾은 게
# 동일한 Order_id 이지만, 여러 종류의 제품을 담은 것을 볼 수 있었음
order_items.loc[order_items['Order_id']=='ORDER_87975']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
97896,ORDER_87975,1,PRODUCT_28697,SELLER_0417,49.9,24.37
97897,ORDER_87975,2,PRODUCT_12522,SELLER_0417,49.9,6.09
97898,ORDER_87975,3,PRODUCT_28697,SELLER_0417,49.9,24.37
97899,ORDER_87975,4,PRODUCT_28697,SELLER_0417,49.9,24.37
97900,ORDER_87975,5,PRODUCT_28697,SELLER_0417,49.9,24.37


In [28]:
# 헤당 주문에 대한 리뷰는 1개만 작성되어 있었음..
reviews.loc[reviews['Order_id']=='ORDER_87975']

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
48859,REVIEW_48713,ORDER_87975,5,2019-06-08 00:00:00,2019-06-09 02:17:23


In [32]:
# 이런 경우가 있을 것을 고려해서 reviews 에 다른 csv 파일을 붙이기보다는, order_items 에 리뷰 점수를 붙이는 게 맞다고 생각
# reviews에서 Order_id 만 일치할 경우 평균 값으로 합산하는 과정을 먼저 거치고 (Review_id 컬럼 무시)
# order_items 에 Order_id 가 일치하면 위 과정을 완료한 reviews 의  Review_score 를 부여하는 방향으로 진행 예정

In [7]:
# 리뷰 아이디가 겹치는 것 확인
reviews.duplicated(['Review_id']).value_counts()

False    87173
True       700
Name: count, dtype: int64

In [9]:
reviews[reviews.duplicated(['Review_id'])]

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
2949,REVIEW_00906,ORDER_43215,5,2018-07-21 00:00:00,2018-07-22 01:06:37
5100,REVIEW_04063,ORDER_03377,5,2018-09-07 00:00:00,2018-09-11 09:58:09
6430,REVIEW_04212,ORDER_52489,5,2018-05-30 00:00:00,2018-06-02 18:16:17
8015,REVIEW_01775,ORDER_66738,5,2018-12-15 00:00:00,2018-12-16 01:32:18
8786,REVIEW_01724,ORDER_84255,5,2019-02-24 00:00:00,2019-02-27 23:09:12
...,...,...,...,...,...
87768,REVIEW_06134,ORDER_09043,4,2018-07-03 00:00:00,2018-07-05 19:06:59
87782,REVIEW_00743,ORDER_85522,5,2018-10-07 00:00:00,2018-10-07 19:47:11
87817,REVIEW_37828,ORDER_44549,5,2019-03-03 00:00:00,2019-03-04 22:56:47
87820,REVIEW_48416,ORDER_46934,5,2019-03-02 00:00:00,2019-03-05 01:43:30


In [30]:
reviews.loc[reviews['Review_id']=='REVIEW_64145']
# Review_id 가 같은데 Order_id 가 다름

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
64420,REVIEW_64145,ORDER_08848,4,2018-05-24 00:00:00,2018-05-24 23:15:21
87830,REVIEW_64145,ORDER_01992,4,2018-05-24 00:00:00,2018-05-24 23:15:21


In [58]:
# order_items 에서 Order_id 마다 제품 구매 내역 보고 (1)
# order 에서 Order_id 마다 고객 아이디 및 유니크 아이디까지 봐야할 듯 (2)

In [36]:
# order_items 에서 Order_id 마다 제품 구매 내역 보고 (1)
order_items.loc[order_items['Order_id']=='ORDER_08848']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
63377,ORDER_08848,1,PRODUCT_21118,SELLER_2101,27.9,15.1


In [37]:
order_items.loc[order_items['Order_id']=='ORDER_01992']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
65784,ORDER_01992,1,PRODUCT_05929,SELLER_1678,89.9,17.88


In [38]:
# 서로 다른 제품을 구매했는데 리뷰 아이디가 동일한 것으로 분류가 되어 있음..

In [57]:
# order 에서 Order_id 마다 고객 아이디 및 유니크 아이디까지 봐야할 듯 (2)
orders.loc[orders['Order_id']=='ORDER_08848']

,Order_id,Customer_id,Order_status,Order_purchase_timestamp,Order_delivered_carrier_date,Order_delivered_customer_date,Order_estimated_delivery_date
8833,ORDER_08848,CUSTOMER_46781,delivered,2018-05-16 12:28:33,2018-05-17 14:09:36,2018-05-22 16:52:23,2018-06-07


In [43]:
orders.loc[orders['Order_id']=='ORDER_01992']

,Order_id,Customer_id,Order_status,Order_purchase_timestamp,Order_delivered_carrier_date,Order_delivered_customer_date,Order_estimated_delivery_date
1988,ORDER_01992,CUSTOMER_78994,delivered,2018-05-16 12:28:32,2018-05-17 13:53:58,2018-05-23 11:53:40,2018-06-16


In [44]:
# 일단 둘의 Customer_id 는 동일함
# customers 에서 Customer_id 조회해서 Customer_unique_id 일치 여부 확인 하겠음 ..

In [48]:
customers.loc[customers['Customer_id']=='CUSTOMER_46781']

,Customer_id,Customer_unique_id,Customer_zipcode_prefix,Customer_city,Customer_state
46706,CUSTOMER_46781,CUSTOMER_Unique_45949,90220,porto alegre,RS


In [49]:
customers.loc[customers['Customer_id']=='CUSTOMER_78994']

,Customer_id,Customer_unique_id,Customer_zipcode_prefix,Customer_city,Customer_state
78873,CUSTOMER_78994,CUSTOMER_Unique_45949,90220,porto alegre,RS


In [53]:
# customers 에서 Customer_unique_id 조회 결과 같은 사람이었음 ...
# Customer_unique_id 가 같으면 리뷰 아이디도 같아지는가를 확인하기 위해 다른 리뷰 아이디로 조회할 것..

In [59]:
reviews.loc[reviews['Review_id']=='REVIEW_00906']

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
905,REVIEW_00906,ORDER_67566,5,2018-07-21 00:00:00,2018-07-22 01:06:37
2949,REVIEW_00906,ORDER_43215,5,2018-07-21 00:00:00,2018-07-22 01:06:37


In [62]:
orders.loc[orders['Order_id']=='ORDER_67566']

,Order_id,Customer_id,Order_status,Order_purchase_timestamp,Order_delivered_carrier_date,Order_delivered_customer_date,Order_estimated_delivery_date
67462,ORDER_67566,CUSTOMER_02658,delivered,2018-07-08 14:55:53,2018-07-12 21:49:31,2018-07-20 17:51:38,2018-08-01


In [63]:
orders.loc[orders['Order_id']=='ORDER_43215']

,Order_id,Customer_id,Order_status,Order_purchase_timestamp,Order_delivered_carrier_date,Order_delivered_customer_date,Order_estimated_delivery_date
43143,ORDER_43215,CUSTOMER_22203,delivered,2018-07-08 14:55:52,2018-07-11 17:12:41,2018-07-19 20:46:57,2018-08-03


In [64]:
customers.loc[orders['Customer_id']=='CUSTOMER_02658']

,Customer_id,Customer_unique_id,Customer_zipcode_prefix,Customer_city,Customer_state
67462,CUSTOMER_67573,CUSTOMER_Unique_65868,32670,betim,MG


In [65]:
customers.loc[orders['Customer_id']=='CUSTOMER_22203']

,Customer_id,Customer_unique_id,Customer_zipcode_prefix,Customer_city,Customer_state
43143,CUSTOMER_43208,CUSTOMER_Unique_42497,6246,osasco,SP


In [67]:
# Review_id 는 같으나, Customer_unique_id 가 다른 경우도 있음...
# 그냥 Review_id 는 무시하는 게 맞을 것 같음 ...

### reviews 에서 Order_id 만 일치할 경우 평균 값으로 Review_score 지정하기

In [72]:
# reviews 에서 필요한 건 Order_id 와 Review_score 이므로 이 둘만 따로 뺀 것을
# re_sc 라 칭하겠음

In [71]:
re_sc = reviews[['Order_id', 'Review_score']]

In [74]:
re_sc

,Order_id,Review_score
0,ORDER_01674,4
1,ORDER_80140,5
2,ORDER_69816,5
3,ORDER_24398,5
4,ORDER_70366,5
...,...,...
87868,ORDER_58840,5
87869,ORDER_75162,5
87870,ORDER_08690,5
87871,ORDER_25681,4


In [78]:
# re_sc 를 평균 구한 것으로 재정의
re_sc = re_sc.groupby(['Order_id']).mean().reset_index()

In [76]:
reviews.loc[reviews['Order_id']=='ORDER_14054']

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
4245,REVIEW_04251,ORDER_14054,2,2018-09-19 00:00:00,2018-09-22 15:23:29
8712,REVIEW_08722,ORDER_14054,3,2018-09-14 00:00:00,2018-09-17 11:14:08


In [81]:
# 임의의 Order_id 를 호출해서 제대로 실행 되었는가 확인
re_sc.loc[re_sc['Order_id']=='ORDER_14054']

,Order_id,Review_score
13956,ORDER_14054,2.5


In [83]:
# 이제 Order_id 당 Review_score 는 re_sc 로 해결 완료!

### order_items 에서 Order_id가 일치하는 여러 개의 행이 있었음

In [86]:
order_items

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69
...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72


In [88]:
# order_items 겹치는 것 확인
order_items.duplicated(['Order_id']).value_counts()

False    87954
True     12603
Name: count, dtype: int64

In [89]:
order_items[order_items.duplicated(['Order_id'])]

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
12,ORDER_47740,2,PRODUCT_04082,SELLER_0417,49.90,13.37
30,ORDER_32079,2,PRODUCT_19082,SELLER_0199,21.33,15.10
31,ORDER_32079,3,PRODUCT_19082,SELLER_0199,21.33,15.10
40,ORDER_23881,2,PRODUCT_23679,SELLER_2076,24.89,17.63
41,ORDER_23881,3,PRODUCT_23679,SELLER_2076,24.89,17.63
...,...,...,...,...,...,...
100525,ORDER_06151,3,PRODUCT_14791,SELLER_0290,21.90,15.79
100542,ORDER_75386,2,PRODUCT_14224,SELLER_1501,180.00,48.14
100548,ORDER_54839,2,PRODUCT_06372,SELLER_2054,55.00,34.19
100549,ORDER_54839,3,PRODUCT_06372,SELLER_2054,55.00,34.19


In [98]:
# 동일 Order_id 내 동일 Product_id 다수 구매한 건 예시
order_items.loc[order_items['Order_id']=='ORDER_47740']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
11,ORDER_47740,1,PRODUCT_04082,SELLER_0417,49.9,13.37
12,ORDER_47740,2,PRODUCT_04082,SELLER_0417,49.9,13.37


In [95]:
# 이제 이게 동일 Order_id 내 Product_id 가 다른 제품 여러 개 구매한 거..
order_items.loc[order_items['Order_id']=='ORDER_87975']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
97896,ORDER_87975,1,PRODUCT_28697,SELLER_0417,49.9,24.37
97897,ORDER_87975,2,PRODUCT_12522,SELLER_0417,49.9,6.09
97898,ORDER_87975,3,PRODUCT_28697,SELLER_0417,49.9,24.37
97899,ORDER_87975,4,PRODUCT_28697,SELLER_0417,49.9,24.37
97900,ORDER_87975,5,PRODUCT_28697,SELLER_0417,49.9,24.37


In [99]:
# order_items 에서는 동일 Order_id 내에서 Product_id 의 갯수를 확인해주는 새로운 컬럼(Counts)을 만들고 싶은데..
# 위 과정을 성공하면 Order_id 와 Product_id 동일할 때 Price , Freight_value , Counts 의 평균으로 합치고 싶음..
# Price , Freight_value , Counts 을 평균으로 하려는 이유는 차피 다 같은 값이니까 평균해도 같은 값 나옴

In [335]:
# 잘못된 행동을 함 Product_id 와 Seller_id 가 동일한 경우 합치는 짓을 해야 했음

In [100]:
order_items['Counts'] = order_items.groupby(['Order_id', 'Product_id'])['Order_item_id'].transform('count')

In [101]:
order_items

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29,1
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93,1
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87,1
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14,1
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69,1
...,...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41,1
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53,1
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95,1
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72,1


In [102]:
order_items.loc[order_items['Order_id']=='ORDER_87975']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
97896,ORDER_87975,1,PRODUCT_28697,SELLER_0417,49.9,24.37,4
97897,ORDER_87975,2,PRODUCT_12522,SELLER_0417,49.9,6.09,1
97898,ORDER_87975,3,PRODUCT_28697,SELLER_0417,49.9,24.37,4
97899,ORDER_87975,4,PRODUCT_28697,SELLER_0417,49.9,24.37,4
97900,ORDER_87975,5,PRODUCT_28697,SELLER_0417,49.9,24.37,4


In [103]:
# order_items 에서는 동일 Order_id 내에서 Product_id 의 갯수를 확인해주는 새로운 컬럼(Counts)을 만들고 싶은데..
# 라고 한 거 성공했다!!!

In [116]:
# 이제 Order_id 랑 Product_id 동일하면 합칠 거임!!!!!!
# Order_item_id 는 Order_id 내에 제품을 담은 순서로 취급하여 해당 값은 무시할 것
# Seller_id 는 안 넣으려니 계속 오류가 나서 넣음 ....
order_items.groupby(['Order_id', 'Product_id', 'Seller_id']).mean().reset_index()

,Order_id,Product_id,Seller_id,Order_item_id,Price,Freight_value,Counts
0,ORDER_00000,PRODUCT_02093,SELLER_0491,1.0,29.99,8.72,1.0
1,ORDER_00001,PRODUCT_26747,SELLER_2662,1.0,45.00,27.20,1.0
2,ORDER_00002,PRODUCT_10598,SELLER_1354,1.0,19.90,8.72,1.0
3,ORDER_00003,PRODUCT_16703,SELLER_1518,1.0,147.90,27.36,1.0
4,ORDER_00004,PRODUCT_08274,SELLER_1599,1.0,59.99,15.17,1.0
...,...,...,...,...,...,...,...
91293,ORDER_88083,PRODUCT_16488,SELLER_2296,1.0,72.00,13.08,1.0
91294,ORDER_88084,PRODUCT_01588,SELLER_0031,1.0,174.90,20.10,1.0
91295,ORDER_88085,PRODUCT_13335,SELLER_1892,1.0,205.99,65.02,1.0
91296,ORDER_88086,PRODUCT_07372,SELLER_2445,1.5,179.99,40.59,2.0


In [122]:
# 근데 뭔가 쎄해서 order_items 에서 Product_id 가 동일한 데 Seller_id 가 다른 것이 있나? 찾아봄 ...
order_items.groupby('Product_id').filter(lambda x: len(set(x['Seller_id'])) > 1)

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
10,ORDER_19931,1,PRODUCT_08596,SELLER_2194,74.00,23.32,1
14,ORDER_86002,1,PRODUCT_27073,SELLER_0891,144.00,8.77,1
21,ORDER_10656,1,PRODUCT_23942,SELLER_0491,48.90,16.60,1
25,ORDER_26578,1,PRODUCT_23242,SELLER_1599,27.90,7.94,1
29,ORDER_32079,1,PRODUCT_19082,SELLER_0199,21.33,15.10,3
...,...,...,...,...,...,...,...
100507,ORDER_13386,1,PRODUCT_14603,SELLER_1257,259.90,27.31,1
100515,ORDER_79011,1,PRODUCT_00703,SELLER_2330,112.90,0.00,1
100522,ORDER_73515,1,PRODUCT_24810,SELLER_0952,47.65,7.39,1
100526,ORDER_86190,1,PRODUCT_12947,SELLER_1132,149.90,16.81,1


In [123]:
# order_items 에서 Product_id 가 동일한 데 Seller_id 가 다른 것이 있었음.....
order_items.loc[order_items['Product_id']=='PRODUCT_08596']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
10,ORDER_19931,1,PRODUCT_08596,SELLER_2194,74.0,23.32,1
277,ORDER_87547,1,PRODUCT_08596,SELLER_2194,79.9,14.65,1
669,ORDER_27319,1,PRODUCT_08596,SELLER_2194,89.9,12.13,2
670,ORDER_27319,2,PRODUCT_08596,SELLER_2194,89.9,12.13,2
983,ORDER_68790,1,PRODUCT_08596,SELLER_2194,89.9,15.38,1
...,...,...,...,...,...,...,...
99465,ORDER_45827,1,PRODUCT_08596,SELLER_2194,79.9,23.36,1
99556,ORDER_04698,1,PRODUCT_08596,SELLER_2194,89.9,15.38,1
99924,ORDER_61863,1,PRODUCT_08596,SELLER_2194,86.9,16.29,1
100173,ORDER_72688,1,PRODUCT_08596,SELLER_2194,89.9,4.63,1


In [120]:
# 그래서 Order_id 는 같은데 Seller_id 가 다른 게 있나 찾아봄..
order_items.groupby('Order_id').filter(lambda x: len(set(x['Seller_id'])) > 1)
# 해보고 생각하니 이건 아까 발견한.. 거였음 왜 했니~~..

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
73,ORDER_67055,1,PRODUCT_21713,SELLER_0065,8.99,32.57,1
74,ORDER_67055,2,PRODUCT_21985,SELLER_0150,44.90,7.16,1
257,ORDER_21853,1,PRODUCT_00629,SELLER_1783,165.50,15.80,1
258,ORDER_21853,2,PRODUCT_07200,SELLER_0629,175.91,52.69,1
259,ORDER_21853,3,PRODUCT_10032,SELLER_1783,165.50,15.80,1
...,...,...,...,...,...,...,...
100244,ORDER_45918,1,PRODUCT_16369,SELLER_1388,50.00,5.29,1
100245,ORDER_45918,2,PRODUCT_11208,SELLER_1974,29.99,26.42,1
100442,ORDER_27491,1,PRODUCT_21899,SELLER_0960,45.00,6.79,1
100443,ORDER_27491,2,PRODUCT_21508,SELLER_2387,79.99,6.37,1


In [ ]:
# 일단은 Seller_id 를 살리고 counts를 넣은 order_items 로 진행할 것임
# Seller_id 를 무시하고 Product_id 의 점수를 비교하는 방향으로 된다면,
# Product_id 를 삭제하고 Price , Freight_value 는 평균 값, Counts 는 합산 값으로 합치면 됨..

# 근데 복잡해질 것 같아서 Seller_id 있는 버전(order_items_ys)과 없는 버전(order_items_ns)을 따로 만들어 두겠습니다 ......... 하하~!
# 여기 과정 잘못 돼서 버리는 거라고 봐야 함

In [150]:
order_items_ys = order_items.groupby(['Order_id', 'Product_id', 'Seller_id']).mean().reset_index()

In [151]:
order_items_ys

,Order_id,Product_id,Seller_id,Order_item_id,Price,Freight_value,Counts
0,ORDER_00000,PRODUCT_02093,SELLER_0491,1.0,29.99,8.72,1.0
1,ORDER_00001,PRODUCT_26747,SELLER_2662,1.0,45.00,27.20,1.0
2,ORDER_00002,PRODUCT_10598,SELLER_1354,1.0,19.90,8.72,1.0
3,ORDER_00003,PRODUCT_16703,SELLER_1518,1.0,147.90,27.36,1.0
4,ORDER_00004,PRODUCT_08274,SELLER_1599,1.0,59.99,15.17,1.0
...,...,...,...,...,...,...,...
91293,ORDER_88083,PRODUCT_16488,SELLER_2296,1.0,72.00,13.08,1.0
91294,ORDER_88084,PRODUCT_01588,SELLER_0031,1.0,174.90,20.10,1.0
91295,ORDER_88085,PRODUCT_13335,SELLER_1892,1.0,205.99,65.02,1.0
91296,ORDER_88086,PRODUCT_07372,SELLER_2445,1.5,179.99,40.59,2.0


In [152]:
order_items_ys['Counts'].value_counts()

Counts
1.0     84875
2.0      4874
3.0       866
4.0       364
5.0       149
6.0       148
10.0        5
7.0         4
12.0        2
8.0         2
9.0         2
15.0        2
14.0        2
20.0        2
13.0        1
Name: count, dtype: int64

In [130]:
order_items_ns = order_items.drop(['Seller_id'], axis=1)

In [198]:
order_items_ns = order_items_ns.groupby(['Order_id', 'Product_id']).mean().reset_index()

In [199]:
order_items_ns['Counts'].value_counts()

Counts
1.0     84875
2.0      4874
3.0       866
4.0       364
5.0       149
6.0       148
10.0        5
7.0         4
12.0        2
8.0         2
9.0         2
15.0        2
14.0        2
20.0        2
13.0        1
Name: count, dtype: int64

In [200]:
order_items['Counts'].value_counts()

Counts
1     84875
2      9748
3      2598
4      1456
6       888
5       745
10       50
20       40
15       30
14       28
7        28
12       24
9        18
8        16
13       13
Name: count, dtype: int64

In [201]:
# value_counts 해서 제대로 들어간 게 맞나 확인하려고..

In [172]:
# 이제 order_items_ys , order_items_ns 각각에
# Order_id 가 동일한 경우 re_sc 에 있는 Review_score 넣으면 됨

# order_items_ys , order_items_ns , re_sc 를 db에 넣겠습니다....
## re_sc 만 살리고 2개는 나중에 새로 넣을 것입니다..

In [159]:
pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [178]:
import pymysql
import pickle

In [182]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, BigInteger, DateTime, Date, Float

In [189]:
con = pymysql.connect(host='172.20.224.1', user='hyun', password='hyun', db='KPI', charset='utf8')
cur = con.cursor()

In [183]:
metadata1 = MetaData()

In [195]:
table1 = Table("order_items_ys", metadata1,
      Column('Order_id', String(100)),
      Column('Product_id', String(100)),
      Column('Seller_id', String(100)),
      Column('Price', Float),
      Column('Freight_value', Float),
      Column('Counts', Float)
     )

InvalidRequestError: Table 'order_items_ys' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [191]:
c

91298

In [203]:
metadata2 = MetaData()

table2 = Table("order_items_ns", metadata2,
      Column('Order_id', String(100)),
      Column('Product_id', String(100)),
      Column('Price', Float),
      Column('Freight_value', Float),
      Column('Counts', Float)
     )

In [204]:
order_items_ns.to_sql("order_items_ns", if_exists='append', con=engine, index=False)

91298

In [206]:
metadata3 = MetaData()

table3 = Table("re_sc", metadata3,
      Column('Order_id', String(100)),
      Column('Review_score', Float)
     )

In [207]:
re_sc.to_sql("re_sc", if_exists='append', con=engine, index=False)

87349

# 이제 order_items_ys 와 order_items_ns 에 Order_id 별 평균 점수 넣는 것을 해야 합니다~
- 나중에
- 그리고
- 어 뭔가 지금 이상한 거 같은데
- 

In [212]:
order_items_ns[order_items_ns.duplicated(['Order_id'])]

,Order_id,Product_id,Order_item_id,Price,Freight_value,Counts
23,ORDER_00022,PRODUCT_12896,2.00,39.50,48.64,1.0
58,ORDER_00056,PRODUCT_07372,1.00,119.00,24.97,1.0
108,ORDER_00105,PRODUCT_14388,2.00,119.99,28.05,1.0
109,ORDER_00105,PRODUCT_16209,1.00,179.90,8.41,1.0
124,ORDER_00119,PRODUCT_23470,2.50,91.99,13.50,2.0
...,...,...,...,...,...,...
91092,ORDER_87885,PRODUCT_11016,2.00,180.00,0.29,1.0
91119,ORDER_87912,PRODUCT_10235,1.00,209.99,15.90,1.0
91183,ORDER_87975,PRODUCT_28697,3.25,49.90,24.37,4.0
91231,ORDER_88022,PRODUCT_16854,1.00,28.50,5.01,1.0


In [213]:
order_items_ns.loc[order_items_ns['Order_id']=='ORDER_00105']

,Order_id,Product_id,Order_item_id,Price,Freight_value,Counts
107,ORDER_00105,PRODUCT_13240,3.0,149.99,32.05,1.0
108,ORDER_00105,PRODUCT_14388,2.0,119.99,28.05,1.0
109,ORDER_00105,PRODUCT_16209,1.0,179.90,8.41,1.0


- 여기에 이제 점수가 들어가,
- 그러면 프로덕트 별 점수로 구분하기로 했지 참

In [214]:
order_items_ns[order_items_ns.duplicated(['Product_id'])]

,Order_id,Product_id,Order_item_id,Price,Freight_value,Counts
63,ORDER_00061,PRODUCT_23242,1.0,27.90,18.30,1.0
118,ORDER_00114,PRODUCT_05174,1.0,89.99,16.39,1.0
177,ORDER_00169,PRODUCT_28734,1.0,59.90,17.13,1.0
179,ORDER_00171,PRODUCT_11967,1.5,69.90,0.00,2.0
200,ORDER_00191,PRODUCT_08714,1.0,159.99,57.79,1.0
...,...,...,...,...,...,...
91291,ORDER_88081,PRODUCT_19267,1.0,370.00,19.43,1.0
91292,ORDER_88082,PRODUCT_07377,1.0,139.90,16.09,1.0
91294,ORDER_88084,PRODUCT_01588,1.0,174.90,20.10,1.0
91295,ORDER_88085,PRODUCT_13335,1.0,205.99,65.02,1.0


In [215]:
order_items_ns.loc[order_items_ns['Product_id']=='PRODUCT_11967']

,Order_id,Product_id,Order_item_id,Price,Freight_value,Counts
42,ORDER_00041,PRODUCT_11967,1.0,69.9,12.43,1.0
179,ORDER_00171,PRODUCT_11967,1.5,69.9,0.00,2.0
204,ORDER_00195,PRODUCT_11967,1.0,69.9,19.98,1.0
586,ORDER_00564,PRODUCT_11967,1.0,69.9,0.00,1.0
675,ORDER_00646,PRODUCT_11967,1.0,69.9,29.90,1.0
...,...,...,...,...,...,...
89544,ORDER_86401,PRODUCT_11967,1.0,69.9,12.43,1.0
89560,ORDER_86417,PRODUCT_11967,1.0,69.9,19.98,1.0
90185,ORDER_87018,PRODUCT_11967,1.0,69.9,29.90,1.0
90475,ORDER_87293,PRODUCT_11967,1.0,69.9,21.57,1.0


아 여기 그러면 카운트를 먼저 합치면 안 됐었네  
카운트 없이 다 있다는 가정 하에 별점을 부여하고  
그 다음에 별점들의 평균을 구하는 새로운 컬럼을 구하고 카운트를 넣어야 했어..

In [225]:
order_items.loc[order_items['Product_id']=='PRODUCT_11967']
# order_items 는 아직 행 합하기 전이네

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
230,ORDER_53900,1,PRODUCT_11967,SELLER_0340,69.9,0.00,1
539,ORDER_28674,1,PRODUCT_11967,SELLER_0340,75.0,20.02,1
563,ORDER_64642,1,PRODUCT_11967,SELLER_0340,69.9,17.40,1
1046,ORDER_06718,1,PRODUCT_11967,SELLER_0340,75.0,13.08,2
1047,ORDER_06718,2,PRODUCT_11967,SELLER_0340,75.0,13.08,2
...,...,...,...,...,...,...,...
99985,ORDER_43772,1,PRODUCT_11967,SELLER_0340,69.9,13.08,1
99990,ORDER_20416,1,PRODUCT_11967,SELLER_0340,69.9,19.98,2
99991,ORDER_20416,2,PRODUCT_11967,SELLER_0340,69.9,19.98,2
100145,ORDER_58033,1,PRODUCT_11967,SELLER_0340,69.9,35.85,1


In [509]:
order_items

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69
...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72


# 일단 셀러 살리는 df(order_items_ys), 없애는 df(order_items_nf) 각자 만들겠음..

In [231]:
order_items_ys = order_items.copy()

In [232]:
order_items_ys

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29,1
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93,1
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87,1
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14,1
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69,1
...,...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41,1
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53,1
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95,1
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72,1


In [234]:
order_items_ns = order_items.drop(['Seller_id'], axis = 1).copy()

In [235]:
order_items_ns

,Order_id,Order_item_id,Product_id,Price,Freight_value,Counts
0,ORDER_75477,1,PRODUCT_23128,58.90,13.29,1
1,ORDER_63604,1,PRODUCT_24351,239.90,19.93,1
2,ORDER_05566,1,PRODUCT_20226,199.00,17.87,1
3,ORDER_04632,1,PRODUCT_07878,199.90,18.14,1
4,ORDER_61452,1,PRODUCT_03505,21.90,12.69,1
...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,299.99,43.41,1
100553,ORDER_62069,1,PRODUCT_08229,350.00,36.53,1
100554,ORDER_46632,1,PRODUCT_21462,99.90,16.95,1
100555,ORDER_52966,1,PRODUCT_19978,55.99,8.72,1


In [239]:
# 둘 다 이 상태로 db에 넣겠음

In [240]:
metadata1 = MetaData()

table1 = Table("order_items_ys", metadata1,
      Column('Order_id', String(100)),
      Column('Product_id', String(100)),
      Column('Seller_id', String(100)),
      Column('Price', Float),
      Column('Freight_value', Float),
      Column('Counts', Float)
     )

In [241]:
order_items_ys.to_sql("order_items_ys", if_exists='append', con=engine, index=False)

100557

In [242]:
metadata2 = MetaData()

table2 = Table("order_items_ns", metadata2,
      Column('Order_id', String(100)),
      Column('Product_id', String(100)),
      Column('Price', Float),
      Column('Freight_value', Float),
      Column('Counts', Float)
     )

In [243]:
order_items_ns.to_sql("order_items_ns", if_exists='append', con=engine, index=False)

100557

## order_items_ys 부터 해볼게

In [237]:
re_sc

,Order_id,Review_score
0,ORDER_00000,4.0
1,ORDER_00001,5.0
2,ORDER_00002,5.0
3,ORDER_00003,4.0
4,ORDER_00004,5.0
...,...,...
87344,ORDER_88083,5.0
87345,ORDER_88084,4.0
87346,ORDER_88085,5.0
87347,ORDER_88086,2.0


In [244]:
# 오더 아이디가 일치하면 점수 넣기
# sql에서 해볼게..

-- order_items_ys 에 리뷰 점수 넣기  
SELECT order_items_ys.* , re_sc.Review_score  
FROM order_items_ys  
INNER JOIN re_sc  
WHERE order_items_ys.Order_id = re_sc.Order_id;  
-- 제품 리뷰가 없으면 행 삭제 되어서 행이 줄어들게 됨

-- 아래 코드는 없으면 0점 되게 하는 거!  
SELECT order_items_ys.* , re_sc.Review_score  
FROM order_items_ys  
LEFT JOIN re_sc  
ON order_items_ys.Order_id = re_sc.Order_id;  

In [245]:
# 일반적으로 리뷰 평점은 총 점수 합/총 리뷰 수 이기 때문에,
# inner join, where 코드를 사용하도록 하겠음!~

In [249]:
#이거 그냥 평균 제대로 되었나 확인 하려고

In [246]:
reviews.loc[reviews['Order_id']=='ORDER_72800']

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
29191,REVIEW_29161,ORDER_72800,4,2019-02-25 00:00:00,2019-02-28 05:19:29
82223,REVIEW_50564,ORDER_72800,1,2019-03-07 00:00:00,2019-03-08 03:00:53


In [248]:
order_items.loc[order_items['Order_id']=='ORDER_72800']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
97920,ORDER_72800,1,PRODUCT_09644,SELLER_1408,61.5,11.93,1


## 리뷰 평점 정리하기
- Seller_id 있는 거 : rs_ys
- Seller_id 없는 거 : rw_ns
- 아래는 db에서 가져오는 과정임

In [251]:
user = 'hyun'
password = 'hyun'
host='172.20.224.1'
port = 3306
database = 'KPI'
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

In [300]:
rs_ys = pd.read_sql_query("SELECT * FROM rs_ys", con=engine)

In [301]:
rs_ns = pd.read_sql_query("SELECT * FROM rs_ns", con=engine)

In [ ]:
sc_ns

### rs_ys 먼저 

In [302]:
rs_ys

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts,Review_score
0,ORDER_00459,1,PRODUCT_12030,SELLER_0983,109.90,25.51,1,5.0
1,ORDER_02214,1,PRODUCT_19385,SELLER_2194,99.00,13.71,1,2.0
2,ORDER_02566,1,PRODUCT_18409,SELLER_1628,249.90,34.23,1,5.0
3,ORDER_02082,1,PRODUCT_13822,SELLER_1059,179.99,85.97,1,5.0
4,ORDER_00164,1,PRODUCT_25591,SELLER_0891,179.90,15.01,2,5.0
...,...,...,...,...,...,...,...,...
99772,ORDER_32269,1,PRODUCT_16426,SELLER_1455,16.90,16.79,1,5.0
99773,ORDER_68888,1,PRODUCT_27140,SELLER_1951,91.80,7.51,1,5.0
99774,ORDER_50126,1,PRODUCT_27917,SELLER_0629,175.90,7.15,1,3.0
99775,ORDER_50126,2,PRODUCT_07200,SELLER_0629,175.91,14.15,1,3.0


In [303]:
# Product_id Seller_id Review_score 만 가져오도록 하겠음..

In [304]:
rs_ys2 = rs_ys.drop(['Order_item_id', 'Freight_value', 'Price', 'Counts'], axis=1)

In [305]:
rs_ys2

,Order_id,Product_id,Seller_id,Review_score
0,ORDER_00459,PRODUCT_12030,SELLER_0983,5.0
1,ORDER_02214,PRODUCT_19385,SELLER_2194,2.0
2,ORDER_02566,PRODUCT_18409,SELLER_1628,5.0
3,ORDER_02082,PRODUCT_13822,SELLER_1059,5.0
4,ORDER_00164,PRODUCT_25591,SELLER_0891,5.0
...,...,...,...,...
99772,ORDER_32269,PRODUCT_16426,SELLER_1455,5.0
99773,ORDER_68888,PRODUCT_27140,SELLER_1951,5.0
99774,ORDER_50126,PRODUCT_27917,SELLER_0629,3.0
99775,ORDER_50126,PRODUCT_07200,SELLER_0629,3.0


In [306]:
rs_ys3 = rs_ys2.groupby(['Order_id', 'Product_id', 'Seller_id']).mean().reset_index()

In [307]:
rs_ys3

,Order_id,Product_id,Seller_id,Review_score
0,ORDER_00000,PRODUCT_02093,SELLER_0491,4.0
1,ORDER_00001,PRODUCT_26747,SELLER_2662,5.0
2,ORDER_00002,PRODUCT_10598,SELLER_1354,5.0
3,ORDER_00003,PRODUCT_16703,SELLER_1518,4.0
4,ORDER_00004,PRODUCT_08274,SELLER_1599,5.0
...,...,...,...,...
90641,ORDER_88083,PRODUCT_16488,SELLER_2296,5.0
90642,ORDER_88084,PRODUCT_01588,SELLER_0031,4.0
90643,ORDER_88085,PRODUCT_13335,SELLER_1892,5.0
90644,ORDER_88086,PRODUCT_07372,SELLER_2445,2.0


In [510]:
rs_ys

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts,Review_score
0,ORDER_00459,1,PRODUCT_12030,SELLER_0983,109.90,25.51,1,5.0
1,ORDER_02214,1,PRODUCT_19385,SELLER_2194,99.00,13.71,1,2.0
2,ORDER_02566,1,PRODUCT_18409,SELLER_1628,249.90,34.23,1,5.0
3,ORDER_02082,1,PRODUCT_13822,SELLER_1059,179.99,85.97,1,5.0
4,ORDER_00164,1,PRODUCT_25591,SELLER_0891,179.90,15.01,2,5.0
...,...,...,...,...,...,...,...,...
99772,ORDER_32269,1,PRODUCT_16426,SELLER_1455,16.90,16.79,1,5.0
99773,ORDER_68888,1,PRODUCT_27140,SELLER_1951,91.80,7.51,1,5.0
99774,ORDER_50126,1,PRODUCT_27917,SELLER_0629,175.90,7.15,1,3.0
99775,ORDER_50126,2,PRODUCT_07200,SELLER_0629,175.91,14.15,1,3.0


In [514]:
rs_ys['Product_id'].duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
99772     True
99773    False
99774     True
99775     True
99776     True
Name: Product_id, Length: 99777, dtype: bool

In [515]:
rs_ys.loc[rs_ys['Product_id']=='PRODUCT_02346']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts,Review_score
49703,ORDER_74393,1,PRODUCT_02346,SELLER_1431,289.0,47.61,1,4.0
67146,ORDER_46639,1,PRODUCT_02346,SELLER_1431,309.0,26.91,1,5.0
99776,ORDER_09101,1,PRODUCT_02346,SELLER_1431,278.0,26.70,1,5.0


In [309]:
rs_ys3 = rs_ys[['Product_id', 'Seller_id', 'Review_score']]

In [310]:
rs_ys3

,Product_id,Seller_id,Review_score
0,PRODUCT_12030,SELLER_0983,5.0
1,PRODUCT_19385,SELLER_2194,2.0
2,PRODUCT_18409,SELLER_1628,5.0
3,PRODUCT_13822,SELLER_1059,5.0
4,PRODUCT_25591,SELLER_0891,5.0
...,...,...,...
99772,PRODUCT_16426,SELLER_1455,5.0
99773,PRODUCT_27140,SELLER_1951,5.0
99774,PRODUCT_27917,SELLER_0629,3.0
99775,PRODUCT_07200,SELLER_0629,3.0


In [321]:
rs_ys4 = rs_ys.groupby(['Product_id', 'Seller_id']).agg({'Review_score' : 'mean'}).reset_index()

In [323]:
rs_ys4
# Product_id 와 Seller_id 가 동일한 리뷰들의 평균 점수 Review_score

,Product_id,Seller_id,Review_score
0,PRODUCT_00000,SELLER_0385,5.000000
1,PRODUCT_00001,SELLER_1601,5.000000
2,PRODUCT_00002,SELLER_0110,5.000000
3,PRODUCT_00003,SELLER_2501,5.000000
4,PRODUCT_00004,SELLER_1294,5.000000
...,...,...,...
30506,PRODUCT_29467,SELLER_0207,5.000000
30507,PRODUCT_29468,SELLER_0768,4.500000
30508,PRODUCT_29468,SELLER_0933,4.666667
30509,PRODUCT_29469,SELLER_0578,4.714286


# 판매 가격을 이제 뽑아야지

In [328]:
# 가격을 이제 뽑아야지
# rs_ys 에서 Product_id , Seller_id , Price , Counts 가져와야 할 듯
# Seller_id 있으면 pr_ys,
# Seller_id 없으면 pr_ns

In [329]:
pr_ys = rs_ys[['Product_id', 'Seller_id', 'Price', 'Counts']]

In [330]:
pr_ys

,Product_id,Seller_id,Price,Counts
0,PRODUCT_12030,SELLER_0983,109.90,1
1,PRODUCT_19385,SELLER_2194,99.00,1
2,PRODUCT_18409,SELLER_1628,249.90,1
3,PRODUCT_13822,SELLER_1059,179.99,1
4,PRODUCT_25591,SELLER_0891,179.90,2
...,...,...,...,...
99772,PRODUCT_16426,SELLER_1455,16.90,1
99773,PRODUCT_27140,SELLER_1951,91.80,1
99774,PRODUCT_27917,SELLER_0629,175.90,1
99775,PRODUCT_07200,SELLER_0629,175.91,1


In [332]:
pr_ys.groupby(['Product_id', 'Seller_id']).mean().reset_index()

,Product_id,Seller_id,Price,Counts
0,PRODUCT_00000,SELLER_0385,10.910000,1.000000
1,PRODUCT_00001,SELLER_1601,248.000000,1.000000
2,PRODUCT_00002,SELLER_0110,79.800000,1.000000
3,PRODUCT_00003,SELLER_2501,37.900000,1.000000
4,PRODUCT_00004,SELLER_1294,45.870000,1.000000
...,...,...,...,...
30506,PRODUCT_29467,SELLER_0207,160.000000,1.000000
30507,PRODUCT_29468,SELLER_0768,127.000000,1.000000
30508,PRODUCT_29468,SELLER_0933,127.500000,1.000000
30509,PRODUCT_29469,SELLER_0578,32.702857,2.428571


In [333]:
pr_ys.loc[pr_ys['Product_id']=='PRODUCT_29469']

,Product_id,Seller_id,Price,Counts
11579,PRODUCT_29469,SELLER_0578,29.90,2
11580,PRODUCT_29469,SELLER_0578,29.90,2
64664,PRODUCT_29469,SELLER_0578,39.71,2
64665,PRODUCT_29469,SELLER_0578,39.71,2
71110,PRODUCT_29469,SELLER_0578,29.90,3
71111,PRODUCT_29469,SELLER_0578,29.90,3
71112,PRODUCT_29469,SELLER_0578,29.90,3


### Counts 오류 이슈로 없는 것으로 다시 재정의 할 것

In [337]:
pr_ys = rs_ys[['Product_id', 'Seller_id', 'Price']]

In [338]:
pr_ys

,Product_id,Seller_id,Price
0,PRODUCT_12030,SELLER_0983,109.90
1,PRODUCT_19385,SELLER_2194,99.00
2,PRODUCT_18409,SELLER_1628,249.90
3,PRODUCT_13822,SELLER_1059,179.99
4,PRODUCT_25591,SELLER_0891,179.90
...,...,...,...
99772,PRODUCT_16426,SELLER_1455,16.90
99773,PRODUCT_27140,SELLER_1951,91.80
99774,PRODUCT_27917,SELLER_0629,175.90
99775,PRODUCT_07200,SELLER_0629,175.91


In [347]:
pr_ys[pr_ys.duplicated('Product_id')]

,Product_id,Seller_id,Price
5,PRODUCT_25591,SELLER_0891,179.90
25,PRODUCT_15510,SELLER_0567,39.49
49,PRODUCT_20715,SELLER_1215,29.00
71,PRODUCT_08694,SELLER_0533,18.99
73,PRODUCT_24628,SELLER_1097,39.90
...,...,...,...
99770,PRODUCT_22785,SELLER_0370,209.99
99772,PRODUCT_16426,SELLER_1455,16.90
99774,PRODUCT_27917,SELLER_0629,175.90
99775,PRODUCT_07200,SELLER_0629,175.91


In [348]:
pr_ys.loc[pr_ys['Product_id']=='PRODUCT_15510']

,Product_id,Seller_id,Price
24,PRODUCT_15510,SELLER_0567,39.49
25,PRODUCT_15510,SELLER_0567,39.49
19998,PRODUCT_15510,SELLER_0567,29.00
69186,PRODUCT_15510,SELLER_0567,34.00
79487,PRODUCT_15510,SELLER_0567,29.00
79488,PRODUCT_15510,SELLER_0567,29.00


In [354]:
pr_ys['Counts'] = pr_ys.groupby(['Product_id', 'Seller_id'])['Price'].transform('count')

/tmp/ipykernel_26834/1926277035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_ys['Counts'] = pr_ys.groupby(['Product_id', 'Seller_id'])['Price'].transform('count')


In [355]:
pr_ys

,Product_id,Seller_id,Price,Counts
0,PRODUCT_12030,SELLER_0983,109.90,1
1,PRODUCT_19385,SELLER_2194,99.00,100
2,PRODUCT_18409,SELLER_1628,249.90,5
3,PRODUCT_13822,SELLER_1059,179.99,2
4,PRODUCT_25591,SELLER_0891,179.90,6
...,...,...,...,...
99772,PRODUCT_16426,SELLER_1455,16.90,2
99773,PRODUCT_27140,SELLER_1951,91.80,1
99774,PRODUCT_27917,SELLER_0629,175.90,6
99775,PRODUCT_07200,SELLER_0629,175.91,8


In [360]:
pr_ys.loc[pr_ys['Product_id']=='PRODUCT_15510']

,Product_id,Seller_id,Price,Counts
24,PRODUCT_15510,SELLER_0567,39.49,6
25,PRODUCT_15510,SELLER_0567,39.49,6
19998,PRODUCT_15510,SELLER_0567,29.00,6
69186,PRODUCT_15510,SELLER_0567,34.00,6
79487,PRODUCT_15510,SELLER_0567,29.00,6
79488,PRODUCT_15510,SELLER_0567,29.00,6


In [361]:
# 이게 진정한 카운츠다 ........
# 근데 왜 Product_id , Seller_id 같은데 Price 다르냐...? ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ

In [363]:
# 위에서 추가한 Counts 드랍하고 셋다 동일한 경우를 카운트 하는 방향으로 다시 잡겠습니다
# Product_id 랑 Seller_id 만 같아도 걍 동일 제품 취급할게요 ...

In [368]:
pr_ys = pr_ys.drop(columns=['Counts'])

In [369]:
pr_ys

,Product_id,Seller_id,Price
0,PRODUCT_12030,SELLER_0983,109.90
1,PRODUCT_19385,SELLER_2194,99.00
2,PRODUCT_18409,SELLER_1628,249.90
3,PRODUCT_13822,SELLER_1059,179.99
4,PRODUCT_25591,SELLER_0891,179.90
...,...,...,...
99772,PRODUCT_16426,SELLER_1455,16.90
99773,PRODUCT_27140,SELLER_1951,91.80
99774,PRODUCT_27917,SELLER_0629,175.90
99775,PRODUCT_07200,SELLER_0629,175.91


In [373]:
pr_ys

,Product_id,Seller_id,Price
0,PRODUCT_12030,SELLER_0983,109.90
1,PRODUCT_19385,SELLER_2194,99.00
2,PRODUCT_18409,SELLER_1628,249.90
3,PRODUCT_13822,SELLER_1059,179.99
4,PRODUCT_25591,SELLER_0891,179.90
...,...,...,...
99772,PRODUCT_16426,SELLER_1455,16.90
99773,PRODUCT_27140,SELLER_1951,91.80
99774,PRODUCT_27917,SELLER_0629,175.90
99775,PRODUCT_07200,SELLER_0629,175.91


In [445]:
# Product_id Seller_id Price 모두 동일한 숫자를 세는 컬럼을 추가 해
# 그리고 가격*수(총 금액) 를 한 컬럼을 붙여
# 그리고 Product_id Seller_id 같으면 합쳐
# 자연스럽게 총 금액도 합쳐짐

In [377]:
pr_ys.groupby(['Product_id', 'Seller_id', 'Price'])['Product_id'].transform('count')

0         1
1        44
2         2
3         1
4         6
         ..
99772     2
99773     1
99774     4
99775     8
99776     1
Name: Product_id, Length: 99777, dtype: int64

In [399]:
pr_ys.loc[(pr_ys['Product_id'] == 'PRODUCT_15510') & (pr_ys['Seller_id'] == 'SELLER_0567') & (pr_ys['Price'] == '39.49')]

,Product_id,Seller_id,Price


In [398]:
pr_ys.loc[pr_ys['Product_id']=='PRODUCT_15510']

,Product_id,Seller_id,Price
24,PRODUCT_15510,SELLER_0567,39.49
25,PRODUCT_15510,SELLER_0567,39.49
19998,PRODUCT_15510,SELLER_0567,29.00
69186,PRODUCT_15510,SELLER_0567,34.00
79487,PRODUCT_15510,SELLER_0567,29.00
79488,PRODUCT_15510,SELLER_0567,29.00


In [400]:
order_items.loc[order_items['Product_id']=='PRODUCT_15510']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
574,ORDER_00113,1,PRODUCT_15510,SELLER_0567,39.49,8.27,2
575,ORDER_00113,2,PRODUCT_15510,SELLER_0567,39.49,8.27,2
15150,ORDER_27506,1,PRODUCT_15510,SELLER_0567,29.00,20.80,1
21978,ORDER_07689,1,PRODUCT_15510,SELLER_0567,29.00,10.96,2
21979,ORDER_07689,2,PRODUCT_15510,SELLER_0567,29.00,10.96,2
77498,ORDER_43760,1,PRODUCT_15510,SELLER_0567,34.00,16.79,1


In [401]:
pr_ys = rs_ys[['Order_id', 'Product_id', 'Seller_id', 'Price']]

In [402]:
pr_ys

,Order_id,Product_id,Seller_id,Price
0,ORDER_00459,PRODUCT_12030,SELLER_0983,109.90
1,ORDER_02214,PRODUCT_19385,SELLER_2194,99.00
2,ORDER_02566,PRODUCT_18409,SELLER_1628,249.90
3,ORDER_02082,PRODUCT_13822,SELLER_1059,179.99
4,ORDER_00164,PRODUCT_25591,SELLER_0891,179.90
...,...,...,...,...
99772,ORDER_32269,PRODUCT_16426,SELLER_1455,16.90
99773,ORDER_68888,PRODUCT_27140,SELLER_1951,91.80
99774,ORDER_50126,PRODUCT_27917,SELLER_0629,175.90
99775,ORDER_50126,PRODUCT_07200,SELLER_0629,175.91


In [403]:
pr_ys.loc[pr_ys['Product_id']=='PRODUCT_15510']

,Order_id,Product_id,Seller_id,Price
24,ORDER_00113,PRODUCT_15510,SELLER_0567,39.49
25,ORDER_00113,PRODUCT_15510,SELLER_0567,39.49
19998,ORDER_27506,PRODUCT_15510,SELLER_0567,29.00
69186,ORDER_43760,PRODUCT_15510,SELLER_0567,34.00
79487,ORDER_07689,PRODUCT_15510,SELLER_0567,29.00
79488,ORDER_07689,PRODUCT_15510,SELLER_0567,29.00


In [404]:
order_items.loc[order_items['Product_id']=='PRODUCT_15510']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value,Counts
574,ORDER_00113,1,PRODUCT_15510,SELLER_0567,39.49,8.27,2
575,ORDER_00113,2,PRODUCT_15510,SELLER_0567,39.49,8.27,2
15150,ORDER_27506,1,PRODUCT_15510,SELLER_0567,29.00,20.80,1
21978,ORDER_07689,1,PRODUCT_15510,SELLER_0567,29.00,10.96,2
21979,ORDER_07689,2,PRODUCT_15510,SELLER_0567,29.00,10.96,2
77498,ORDER_43760,1,PRODUCT_15510,SELLER_0567,34.00,16.79,1


In [407]:
order_items = order_items.drop(columns='Counts')

In [408]:
order_items

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69
...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72


In [409]:
pr_ys

,Order_id,Product_id,Seller_id,Price
0,ORDER_00459,PRODUCT_12030,SELLER_0983,109.90
1,ORDER_02214,PRODUCT_19385,SELLER_2194,99.00
2,ORDER_02566,PRODUCT_18409,SELLER_1628,249.90
3,ORDER_02082,PRODUCT_13822,SELLER_1059,179.99
4,ORDER_00164,PRODUCT_25591,SELLER_0891,179.90
...,...,...,...,...
99772,ORDER_32269,PRODUCT_16426,SELLER_1455,16.90
99773,ORDER_68888,PRODUCT_27140,SELLER_1951,91.80
99774,ORDER_50126,PRODUCT_27917,SELLER_0629,175.90
99775,ORDER_50126,PRODUCT_07200,SELLER_0629,175.91


In [412]:
pr_ys['Row_Number'] = pr_ys.reset_index().index + 1

/tmp/ipykernel_26834/265097144.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_ys['Row_Number'] = pr_ys.reset_index().index + 1


In [413]:
pr_ys

,Order_id,Product_id,Seller_id,Price,Row_Number
0,ORDER_00459,PRODUCT_12030,SELLER_0983,109.90,1
1,ORDER_02214,PRODUCT_19385,SELLER_2194,99.00,2
2,ORDER_02566,PRODUCT_18409,SELLER_1628,249.90,3
3,ORDER_02082,PRODUCT_13822,SELLER_1059,179.99,4
4,ORDER_00164,PRODUCT_25591,SELLER_0891,179.90,5
...,...,...,...,...,...
99772,ORDER_32269,PRODUCT_16426,SELLER_1455,16.90,99773
99773,ORDER_68888,PRODUCT_27140,SELLER_1951,91.80,99774
99774,ORDER_50126,PRODUCT_27917,SELLER_0629,175.90,99775
99775,ORDER_50126,PRODUCT_07200,SELLER_0629,175.91,99776


In [414]:
pr_ys['Counts'] = pr_ys.groupby(['Order_id', 'Product_id', 'Seller_id', 'Price'])['Row_Number'].transform('count')

/tmp/ipykernel_26834/1889140404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_ys['Counts'] = pr_ys.groupby(['Order_id', 'Product_id', 'Seller_id', 'Price'])['Row_Number'].transform('count')


In [418]:
pr_ys
# 이건 오더 아이디, 프로덕트 아이디, 셀러 아이디, 가격이 같은 경우 행값 추가한 거임
# 로우 넘거는 의미 없고, 행 숫자 셀 때 필요한 애라서 넣었음!

,Order_id,Product_id,Seller_id,Price,Row_Number,Counts
0,ORDER_00459,PRODUCT_12030,SELLER_0983,109.90,1,1
1,ORDER_02214,PRODUCT_19385,SELLER_2194,99.00,2,1
2,ORDER_02566,PRODUCT_18409,SELLER_1628,249.90,3,1
3,ORDER_02082,PRODUCT_13822,SELLER_1059,179.99,4,1
4,ORDER_00164,PRODUCT_25591,SELLER_0891,179.90,5,2
...,...,...,...,...,...,...
99772,ORDER_32269,PRODUCT_16426,SELLER_1455,16.90,99773,1
99773,ORDER_68888,PRODUCT_27140,SELLER_1951,91.80,99774,1
99774,ORDER_50126,PRODUCT_27917,SELLER_0629,175.90,99775,1
99775,ORDER_50126,PRODUCT_07200,SELLER_0629,175.91,99776,1


In [518]:
pr_ys

,Order_id,Product_id,Seller_id,Price,Row_Number,Counts
0,ORDER_00459,PRODUCT_12030,SELLER_0983,109.90,1,1
1,ORDER_02214,PRODUCT_19385,SELLER_2194,99.00,2,1
2,ORDER_02566,PRODUCT_18409,SELLER_1628,249.90,3,1
3,ORDER_02082,PRODUCT_13822,SELLER_1059,179.99,4,1
4,ORDER_00164,PRODUCT_25591,SELLER_0891,179.90,5,2
...,...,...,...,...,...,...
99772,ORDER_32269,PRODUCT_16426,SELLER_1455,16.90,99773,1
99773,ORDER_68888,PRODUCT_27140,SELLER_1951,91.80,99774,1
99774,ORDER_50126,PRODUCT_27917,SELLER_0629,175.90,99775,1
99775,ORDER_50126,PRODUCT_07200,SELLER_0629,175.91,99776,1


In [517]:
pr_ys.loc[pr_ys['Product_id']=='PRODUCT_18409']

,Order_id,Product_id,Seller_id,Price,Row_Number,Counts
2,ORDER_02566,PRODUCT_18409,SELLER_1628,249.9,3,1
18441,ORDER_18470,PRODUCT_18409,SELLER_1628,249.9,18442,1
29880,ORDER_05073,PRODUCT_18409,SELLER_1628,269.9,29881,1
41036,ORDER_85746,PRODUCT_18409,SELLER_1628,295.0,41037,1
58913,ORDER_54865,PRODUCT_18409,SELLER_1628,249.0,58914,1


In [416]:
pr_ys.loc[pr_ys['Product_id']=='PRODUCT_15510']

,Order_id,Product_id,Seller_id,Price,Row_Number,Counts
24,ORDER_00113,PRODUCT_15510,SELLER_0567,39.49,25,2
25,ORDER_00113,PRODUCT_15510,SELLER_0567,39.49,26,2
19998,ORDER_27506,PRODUCT_15510,SELLER_0567,29.00,19999,1
69186,ORDER_43760,PRODUCT_15510,SELLER_0567,34.00,69187,1
79487,ORDER_07689,PRODUCT_15510,SELLER_0567,29.00,79488,2
79488,ORDER_07689,PRODUCT_15510,SELLER_0567,29.00,79489,2


In [419]:
pr_ys

,Order_id,Product_id,Seller_id,Price,Row_Number,Counts
0,ORDER_00459,PRODUCT_12030,SELLER_0983,109.90,1,1
1,ORDER_02214,PRODUCT_19385,SELLER_2194,99.00,2,1
2,ORDER_02566,PRODUCT_18409,SELLER_1628,249.90,3,1
3,ORDER_02082,PRODUCT_13822,SELLER_1059,179.99,4,1
4,ORDER_00164,PRODUCT_25591,SELLER_0891,179.90,5,2
...,...,...,...,...,...,...
99772,ORDER_32269,PRODUCT_16426,SELLER_1455,16.90,99773,1
99773,ORDER_68888,PRODUCT_27140,SELLER_1951,91.80,99774,1
99774,ORDER_50126,PRODUCT_27917,SELLER_0629,175.90,99775,1
99775,ORDER_50126,PRODUCT_07200,SELLER_0629,175.91,99776,1


In [425]:
pr_ys2 = pr_ys.groupby(['Order_id', 'Product_id', 'Seller_id', 'Price', 'Counts'], as_index=False)['Row_Number'].sum()

In [426]:
pr_ys2

,Order_id,Product_id,Seller_id,Price,Counts,Row_Number
0,ORDER_00000,PRODUCT_02093,SELLER_0491,29.99,1,4168
1,ORDER_00001,PRODUCT_26747,SELLER_2662,45.00,1,2686
2,ORDER_00002,PRODUCT_10598,SELLER_1354,19.90,1,3152
3,ORDER_00003,PRODUCT_16703,SELLER_1518,147.90,1,2982
4,ORDER_00004,PRODUCT_08274,SELLER_1599,59.99,1,1794
...,...,...,...,...,...,...
90641,ORDER_88083,PRODUCT_16488,SELLER_2296,72.00,1,59773
90642,ORDER_88084,PRODUCT_01588,SELLER_0031,174.90,1,14677
90643,ORDER_88085,PRODUCT_13335,SELLER_1892,205.99,1,6210
90644,ORDER_88086,PRODUCT_07372,SELLER_2445,179.99,2,62895


In [427]:
pr_ys2.loc[pr_ys2['Product_id']=='PRODUCT_15510']

,Order_id,Product_id,Seller_id,Price,Counts,Row_Number
116,ORDER_00113,PRODUCT_15510,SELLER_0567,39.49,2,51
7927,ORDER_07689,PRODUCT_15510,SELLER_0567,29.00,2,158977
28288,ORDER_27506,PRODUCT_15510,SELLER_0567,29.00,1,19999
44995,ORDER_43760,PRODUCT_15510,SELLER_0567,34.00,1,69187


In [429]:
pr_ys3 = pr_ys2.drop(columns='Row_Number')

In [430]:
pr_ys3
# Order_id	Product_id	Seller_id	Price 동일한 애들 숫자로 Counts 넣은 거!

,Order_id,Product_id,Seller_id,Price,Counts
0,ORDER_00000,PRODUCT_02093,SELLER_0491,29.99,1
1,ORDER_00001,PRODUCT_26747,SELLER_2662,45.00,1
2,ORDER_00002,PRODUCT_10598,SELLER_1354,19.90,1
3,ORDER_00003,PRODUCT_16703,SELLER_1518,147.90,1
4,ORDER_00004,PRODUCT_08274,SELLER_1599,59.99,1
...,...,...,...,...,...
90641,ORDER_88083,PRODUCT_16488,SELLER_2296,72.00,1
90642,ORDER_88084,PRODUCT_01588,SELLER_0031,174.90,1
90643,ORDER_88085,PRODUCT_13335,SELLER_1892,205.99,1
90644,ORDER_88086,PRODUCT_07372,SELLER_2445,179.99,2


In [431]:
pr_ys3.loc[pr_ys3['Product_id']=='PRODUCT_15510']

,Order_id,Product_id,Seller_id,Price,Counts
116,ORDER_00113,PRODUCT_15510,SELLER_0567,39.49,2
7927,ORDER_07689,PRODUCT_15510,SELLER_0567,29.00,2
28288,ORDER_27506,PRODUCT_15510,SELLER_0567,29.00,1
44995,ORDER_43760,PRODUCT_15510,SELLER_0567,34.00,1


In [432]:
order_items.loc[order_items['Product_id']=='PRODUCT_15510']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
574,ORDER_00113,1,PRODUCT_15510,SELLER_0567,39.49,8.27
575,ORDER_00113,2,PRODUCT_15510,SELLER_0567,39.49,8.27
15150,ORDER_27506,1,PRODUCT_15510,SELLER_0567,29.00,20.80
21978,ORDER_07689,1,PRODUCT_15510,SELLER_0567,29.00,10.96
21979,ORDER_07689,2,PRODUCT_15510,SELLER_0567,29.00,10.96
77498,ORDER_43760,1,PRODUCT_15510,SELLER_0567,34.00,16.79


In [433]:
# 아차차 ... Order_item_id	Product_id	Seller_id 같으면 합치면 됐네 ...

In [434]:
# 아 근데 뭐 이상한게 있을 까 보려고 한 거라서 ㄱㅊㄱㅊ

In [520]:
pr_ys3

,Order_id,Product_id,Seller_id,Price,Counts
0,ORDER_00000,PRODUCT_02093,SELLER_0491,29.99,1
1,ORDER_00001,PRODUCT_26747,SELLER_2662,45.00,1
2,ORDER_00002,PRODUCT_10598,SELLER_1354,19.90,1
3,ORDER_00003,PRODUCT_16703,SELLER_1518,147.90,1
4,ORDER_00004,PRODUCT_08274,SELLER_1599,59.99,1
...,...,...,...,...,...
90641,ORDER_88083,PRODUCT_16488,SELLER_2296,72.00,1
90642,ORDER_88084,PRODUCT_01588,SELLER_0031,174.90,1
90643,ORDER_88085,PRODUCT_13335,SELLER_1892,205.99,1
90644,ORDER_88086,PRODUCT_07372,SELLER_2445,179.99,2


In [437]:
pr_ys4 = pr_ys3.drop(columns='Order_id')

In [519]:
pr_ys4

,Product_id,Seller_id,Price,Counts
0,PRODUCT_02093,SELLER_0491,29.99,1
1,PRODUCT_26747,SELLER_2662,45.00,1
2,PRODUCT_10598,SELLER_1354,19.90,1
3,PRODUCT_16703,SELLER_1518,147.90,1
4,PRODUCT_08274,SELLER_1599,59.99,1
...,...,...,...,...
90641,PRODUCT_16488,SELLER_2296,72.00,1
90642,PRODUCT_01588,SELLER_0031,174.90,1
90643,PRODUCT_13335,SELLER_1892,205.99,1
90644,PRODUCT_07372,SELLER_2445,179.99,2


In [441]:
pr_ys5 = pr_ys4.groupby(['Product_id', 'Seller_id', 'Price']).sum().reset_index()

In [442]:
pr_ys5

,Product_id,Seller_id,Price,Counts
0,PRODUCT_00000,SELLER_0385,10.91,1
1,PRODUCT_00001,SELLER_1601,248.00,1
2,PRODUCT_00002,SELLER_0110,79.80,1
3,PRODUCT_00003,SELLER_2501,37.90,1
4,PRODUCT_00004,SELLER_1294,45.87,1
...,...,...,...,...
37586,PRODUCT_29468,SELLER_0768,127.00,2
37587,PRODUCT_29468,SELLER_0933,127.50,3
37588,PRODUCT_29469,SELLER_0578,29.90,5
37589,PRODUCT_29469,SELLER_0578,39.71,2


In [443]:
pr_ys5.loc[pr_ys5['Product_id']=='PRODUCT_15510']

,Product_id,Seller_id,Price,Counts
19725,PRODUCT_15510,SELLER_0567,29.00,3
19726,PRODUCT_15510,SELLER_0567,34.00,1
19727,PRODUCT_15510,SELLER_0567,39.49,2


In [446]:
# 지금까지 한 건 주문 내역 중에 Product_id	Seller_id	Price 같은 거 Counts 에 행 수 넣은 것임
# 이제 가격 * 갯수 한 거 넣어야 함(total)
# 그리고 나서 Product_id Seller_id 같은 거 합치게 하고(그럼 Counts랑 total이 합쳐지겠지?) (가격 합쳐지는 건 걍 무시해라..)
# 그러면 이제 total이 Product_id Seller_id 의 판매액 되는 거임

In [448]:
pr_ys5['Total'] = pr_ys5['Price'] * pr_ys5['Counts']

In [450]:
pr_ys5
# Product_id, Seller_id, Price 가 동시에 같은 걸 센 것이 Counts 이고
# Price * Counts 가 Total
# 이어서 Product_id, Seller_id 같으면 합치는 작업할 예정
# 그럼 총 가격이 나옴!

,Product_id,Seller_id,Price,Counts,Total
0,PRODUCT_00000,SELLER_0385,10.91,1,10.91
1,PRODUCT_00001,SELLER_1601,248.00,1,248.00
2,PRODUCT_00002,SELLER_0110,79.80,1,79.80
3,PRODUCT_00003,SELLER_2501,37.90,1,37.90
4,PRODUCT_00004,SELLER_1294,45.87,1,45.87
...,...,...,...,...,...
37586,PRODUCT_29468,SELLER_0768,127.00,2,254.00
37587,PRODUCT_29468,SELLER_0933,127.50,3,382.50
37588,PRODUCT_29469,SELLER_0578,29.90,5,149.50
37589,PRODUCT_29469,SELLER_0578,39.71,2,79.42


In [491]:
pr_ys6 = pr_ys5.groupby(['Product_id', 'Seller_id']).sum().reset_index()
# rs_ys4 랑 행 수 같음 !!!!!!!!!!!!1

In [492]:
pr_ys6

,Product_id,Seller_id,Price,Counts,Total
0,PRODUCT_00000,SELLER_0385,10.91,1,10.91
1,PRODUCT_00001,SELLER_1601,248.00,1,248.00
2,PRODUCT_00002,SELLER_0110,79.80,1,79.80
3,PRODUCT_00003,SELLER_2501,37.90,1,37.90
4,PRODUCT_00004,SELLER_1294,45.87,1,45.87
...,...,...,...,...,...
30506,PRODUCT_29467,SELLER_0207,160.00,1,160.00
30507,PRODUCT_29468,SELLER_0768,127.00,2,254.00
30508,PRODUCT_29468,SELLER_0933,127.50,3,382.50
30509,PRODUCT_29469,SELLER_0578,69.61,7,228.92


In [493]:
# 필요없는 행 지우겠음
pr_ys6 = pr_ys6.drop(columns = 'Price')

In [494]:
pr_ys6.rename(columns={"Total" : "Total_price"}, inplace=True)

In [495]:
pr_ys6

,Product_id,Seller_id,Counts,Total_price
0,PRODUCT_00000,SELLER_0385,1,10.91
1,PRODUCT_00001,SELLER_1601,1,248.00
2,PRODUCT_00002,SELLER_0110,1,79.80
3,PRODUCT_00003,SELLER_2501,1,37.90
4,PRODUCT_00004,SELLER_1294,1,45.87
...,...,...,...,...
30506,PRODUCT_29467,SELLER_0207,1,160.00
30507,PRODUCT_29468,SELLER_0768,2,254.00
30508,PRODUCT_29468,SELLER_0933,3,382.50
30509,PRODUCT_29469,SELLER_0578,7,228.92


In [498]:
rs_ys4

,Product_id,Seller_id,Review_score
0,PRODUCT_00000,SELLER_0385,5.000000
1,PRODUCT_00001,SELLER_1601,5.000000
2,PRODUCT_00002,SELLER_0110,5.000000
3,PRODUCT_00003,SELLER_2501,5.000000
4,PRODUCT_00004,SELLER_1294,5.000000
...,...,...,...
30506,PRODUCT_29467,SELLER_0207,5.000000
30507,PRODUCT_29468,SELLER_0768,4.500000
30508,PRODUCT_29468,SELLER_0933,4.666667
30509,PRODUCT_29469,SELLER_0578,4.714286


In [499]:
score_price_ys = pd.merge(rs_ys4, pr_ys6, on=['Product_id', 'Seller_id'], how='inner')

In [500]:
score_price_ys

,Product_id,Seller_id,Review_score,Counts,Total_price
0,PRODUCT_00000,SELLER_0385,5.000000,1,10.91
1,PRODUCT_00001,SELLER_1601,5.000000,1,248.00
2,PRODUCT_00002,SELLER_0110,5.000000,1,79.80
3,PRODUCT_00003,SELLER_2501,5.000000,1,37.90
4,PRODUCT_00004,SELLER_1294,5.000000,1,45.87
...,...,...,...,...,...
30506,PRODUCT_29467,SELLER_0207,5.000000,1,160.00
30507,PRODUCT_29468,SELLER_0768,4.500000,2,254.00
30508,PRODUCT_29468,SELLER_0933,4.666667,3,382.50
30509,PRODUCT_29469,SELLER_0578,4.714286,7,228.92


In [608]:
# Seller_id 있을 때 Review_score 랑 Total_price 상관계수
review_ys_1 = score_price_ys['Review_score'].corr(score_price_ys['Total_price'])

In [609]:
review_ys_1

-0.007800808718716167

In [610]:
review_ys_3 = score_price_ys['Review_score'].corr(score_price_ys['Total_price'])

In [611]:
review_ys_3

-0.007800808718716167

In [612]:
# Seller_id 있을 때 Review_score 랑 Counts 상관계수
review_ys_2 = score_price_ys['Review_score'].corr(score_price_ys['Counts'])

In [613]:
review_ys_2

-0.01025935443501701

In [507]:
# Seller_id 있는 버전은 그냥 다 무의미...

# Product_id 만 고려하는 거 이제 시작...

In [522]:
rs_ns

,Order_id,Order_item_id,Product_id,Price,Freight_value,Counts,Review_score
0,ORDER_00459,1,PRODUCT_12030,109.90,25.51,1,5.0
1,ORDER_02214,1,PRODUCT_19385,99.00,13.71,1,2.0
2,ORDER_02566,1,PRODUCT_18409,249.90,34.23,1,5.0
3,ORDER_02082,1,PRODUCT_13822,179.99,85.97,1,5.0
4,ORDER_00164,1,PRODUCT_25591,179.90,15.01,2,5.0
...,...,...,...,...,...,...,...
99772,ORDER_32269,1,PRODUCT_16426,16.90,16.79,1,5.0
99773,ORDER_68888,1,PRODUCT_27140,91.80,7.51,1,5.0
99774,ORDER_50126,1,PRODUCT_27917,175.90,7.15,1,3.0
99775,ORDER_50126,2,PRODUCT_07200,175.91,14.15,1,3.0


In [527]:
rs_ns2 = rs_ns[['Product_id', 'Price', 'Review_score']]

In [528]:
rs_ns2

,Product_id,Price,Review_score
0,PRODUCT_12030,109.90,5.0
1,PRODUCT_19385,99.00,2.0
2,PRODUCT_18409,249.90,5.0
3,PRODUCT_13822,179.99,5.0
4,PRODUCT_25591,179.90,5.0
...,...,...,...
99772,PRODUCT_16426,16.90,5.0
99773,PRODUCT_27140,91.80,5.0
99774,PRODUCT_27917,175.90,3.0
99775,PRODUCT_07200,175.91,3.0


In [534]:
rs_ns3 = rs_ns2.groupby(['Product_id', 'Price']).mean().reset_index()

In [535]:
rs_ns3

,Product_id,Price,Review_score
0,PRODUCT_00000,10.91,5.000000
1,PRODUCT_00001,248.00,5.000000
2,PRODUCT_00002,79.80,5.000000
3,PRODUCT_00003,37.90,5.000000
4,PRODUCT_00004,45.87,5.000000
...,...,...,...
37452,PRODUCT_29468,127.00,4.500000
37453,PRODUCT_29468,127.50,4.666667
37454,PRODUCT_29469,29.90,4.600000
37455,PRODUCT_29469,39.71,5.000000


## 판매금액 보려고~

In [536]:
rs_ns

,Order_id,Order_item_id,Product_id,Price,Freight_value,Counts,Review_score
0,ORDER_00459,1,PRODUCT_12030,109.90,25.51,1,5.0
1,ORDER_02214,1,PRODUCT_19385,99.00,13.71,1,2.0
2,ORDER_02566,1,PRODUCT_18409,249.90,34.23,1,5.0
3,ORDER_02082,1,PRODUCT_13822,179.99,85.97,1,5.0
4,ORDER_00164,1,PRODUCT_25591,179.90,15.01,2,5.0
...,...,...,...,...,...,...,...
99772,ORDER_32269,1,PRODUCT_16426,16.90,16.79,1,5.0
99773,ORDER_68888,1,PRODUCT_27140,91.80,7.51,1,5.0
99774,ORDER_50126,1,PRODUCT_27917,175.90,7.15,1,3.0
99775,ORDER_50126,2,PRODUCT_07200,175.91,14.15,1,3.0


In [547]:
pr_ns = rs_ns[['Product_id', 'Price']]

In [548]:
pr_ns

,Product_id,Price
0,PRODUCT_12030,109.90
1,PRODUCT_19385,99.00
2,PRODUCT_18409,249.90
3,PRODUCT_13822,179.99
4,PRODUCT_25591,179.90
...,...,...
99772,PRODUCT_16426,16.90
99773,PRODUCT_27140,91.80
99774,PRODUCT_27917,175.90
99775,PRODUCT_07200,175.91


In [549]:
pr_ns['Row_Number'] = pr_ns.reset_index().index + 1

/tmp/ipykernel_26834/1958415129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_ns['Row_Number'] = pr_ns.reset_index().index + 1


In [550]:
pr_ns

,Product_id,Price,Row_Number
0,PRODUCT_12030,109.90,1
1,PRODUCT_19385,99.00,2
2,PRODUCT_18409,249.90,3
3,PRODUCT_13822,179.99,4
4,PRODUCT_25591,179.90,5
...,...,...,...
99772,PRODUCT_16426,16.90,99773
99773,PRODUCT_27140,91.80,99774
99774,PRODUCT_27917,175.90,99775
99775,PRODUCT_07200,175.91,99776


In [551]:
pr_ns

,Product_id,Price,Row_Number
0,PRODUCT_12030,109.90,1
1,PRODUCT_19385,99.00,2
2,PRODUCT_18409,249.90,3
3,PRODUCT_13822,179.99,4
4,PRODUCT_25591,179.90,5
...,...,...,...
99772,PRODUCT_16426,16.90,99773
99773,PRODUCT_27140,91.80,99774
99774,PRODUCT_27917,175.90,99775
99775,PRODUCT_07200,175.91,99776


In [556]:
pr_ns['Counts'] = pr_ns.groupby(['Product_id', 'Price'])['Row_Number'].transform('count')

/tmp/ipykernel_26834/3615390925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_ns['Counts'] = pr_ns.groupby(['Product_id', 'Price'])['Row_Number'].transform('count')


In [557]:
pr_ns

,Product_id,Price,Row_Number,Counts
0,PRODUCT_12030,109.90,1,1
1,PRODUCT_19385,99.00,2,44
2,PRODUCT_18409,249.90,3,2
3,PRODUCT_13822,179.99,4,1
4,PRODUCT_25591,179.90,5,6
...,...,...,...,...
99772,PRODUCT_16426,16.90,99773,2
99773,PRODUCT_27140,91.80,99774,1
99774,PRODUCT_27917,175.90,99775,4
99775,PRODUCT_07200,175.91,99776,8


In [559]:
pr_ns.loc[pr_ns['Product_id']=='PRODUCT_07200']

,Product_id,Price,Row_Number,Counts
27777,PRODUCT_07200,175.91,27778,8
30227,PRODUCT_07200,175.91,30228,8
77288,PRODUCT_07200,175.91,77289,8
77751,PRODUCT_07200,175.91,77752,8
87809,PRODUCT_07200,175.91,87810,8
89016,PRODUCT_07200,175.91,89017,8
91977,PRODUCT_07200,160.00,91978,1
95454,PRODUCT_07200,175.91,95455,8
99775,PRODUCT_07200,175.91,99776,8


In [563]:
pr_ns2 = pr_ns.groupby(['Product_id', 'Price', 'Counts']).sum().reset_index()

In [564]:
pr_ns2

,Product_id,Price,Counts,Row_Number
0,PRODUCT_00000,10.91,1,33882
1,PRODUCT_00001,248.00,1,1490
2,PRODUCT_00002,79.80,1,55126
3,PRODUCT_00003,37.90,1,98387
4,PRODUCT_00004,45.87,1,89976
...,...,...,...,...
37452,PRODUCT_29468,127.00,2,163904
37453,PRODUCT_29468,127.50,3,99560
37454,PRODUCT_29469,29.90,5,236497
37455,PRODUCT_29469,39.71,2,129331


In [567]:
pr_ns2.drop(columns='Row_Number', inplace=True)

In [568]:
pr_ns2

,Product_id,Price,Counts
0,PRODUCT_00000,10.91,1
1,PRODUCT_00001,248.00,1
2,PRODUCT_00002,79.80,1
3,PRODUCT_00003,37.90,1
4,PRODUCT_00004,45.87,1
...,...,...,...
37452,PRODUCT_29468,127.00,2
37453,PRODUCT_29468,127.50,3
37454,PRODUCT_29469,29.90,5
37455,PRODUCT_29469,39.71,2


In [569]:
pr_ns2['Total'] = pr_ns2['Price'] * pr_ns2['Counts']

In [570]:
pr_ns2

,Product_id,Price,Counts,Total
0,PRODUCT_00000,10.91,1,10.91
1,PRODUCT_00001,248.00,1,248.00
2,PRODUCT_00002,79.80,1,79.80
3,PRODUCT_00003,37.90,1,37.90
4,PRODUCT_00004,45.87,1,45.87
...,...,...,...,...
37452,PRODUCT_29468,127.00,2,254.00
37453,PRODUCT_29468,127.50,3,382.50
37454,PRODUCT_29469,29.90,5,149.50
37455,PRODUCT_29469,39.71,2,79.42


In [582]:
pr_ns3 = pr_ns2.groupby(['Product_id']).sum().reset_index()

In [583]:
pr_ns3

,Product_id,Price,Counts,Total
0,PRODUCT_00000,10.91,1,10.91
1,PRODUCT_00001,248.00,1,248.00
2,PRODUCT_00002,79.80,1,79.80
3,PRODUCT_00003,37.90,1,37.90
4,PRODUCT_00004,45.87,1,45.87
...,...,...,...,...
29290,PRODUCT_29466,69.90,28,1957.20
29291,PRODUCT_29467,160.00,1,160.00
29292,PRODUCT_29468,254.50,5,636.50
29293,PRODUCT_29469,69.61,7,228.92


# 아 rs_ns 관련 재정의 해야 함

In [574]:
rs_ns3.loc[rs_ns3['Product_id']=='PRODUCT_29469']

,Product_id,Price,Review_score
37454,PRODUCT_29469,29.90,4.6
37455,PRODUCT_29469,39.71,5.0


In [585]:
rs_ns

,Order_id,Order_item_id,Product_id,Price,Freight_value,Counts,Review_score
0,ORDER_00459,1,PRODUCT_12030,109.90,25.51,1,5.0
1,ORDER_02214,1,PRODUCT_19385,99.00,13.71,1,2.0
2,ORDER_02566,1,PRODUCT_18409,249.90,34.23,1,5.0
3,ORDER_02082,1,PRODUCT_13822,179.99,85.97,1,5.0
4,ORDER_00164,1,PRODUCT_25591,179.90,15.01,2,5.0
...,...,...,...,...,...,...,...
99772,ORDER_32269,1,PRODUCT_16426,16.90,16.79,1,5.0
99773,ORDER_68888,1,PRODUCT_27140,91.80,7.51,1,5.0
99774,ORDER_50126,1,PRODUCT_27917,175.90,7.15,1,3.0
99775,ORDER_50126,2,PRODUCT_07200,175.91,14.15,1,3.0


In [587]:
rs_ns2 = rs_ns[['Product_id', 'Review_score']]

In [588]:
rs_ns2

,Product_id,Review_score
0,PRODUCT_12030,5.0
1,PRODUCT_19385,2.0
2,PRODUCT_18409,5.0
3,PRODUCT_13822,5.0
4,PRODUCT_25591,5.0
...,...,...
99772,PRODUCT_16426,5.0
99773,PRODUCT_27140,5.0
99774,PRODUCT_27917,3.0
99775,PRODUCT_07200,3.0


In [592]:
rs_ns3 = rs_ns2.groupby(['Product_id']).mean().reset_index()

In [593]:
rs_ns3

,Product_id,Review_score
0,PRODUCT_00000,5.000000
1,PRODUCT_00001,5.000000
2,PRODUCT_00002,5.000000
3,PRODUCT_00003,5.000000
4,PRODUCT_00004,5.000000
...,...,...
29290,PRODUCT_29466,4.035714
29291,PRODUCT_29467,5.000000
29292,PRODUCT_29468,4.600000
29293,PRODUCT_29469,4.714286


In [590]:
rs_ns2.loc[rs_ns2['Product_id']=='PRODUCT_29468']

,Product_id,Review_score
1379,PRODUCT_29468,5.0
5980,PRODUCT_29468,4.0
76144,PRODUCT_29468,4.0
87758,PRODUCT_29468,5.0
92198,PRODUCT_29468,5.0


In [595]:
rs_ns3

,Product_id,Review_score
0,PRODUCT_00000,5.000000
1,PRODUCT_00001,5.000000
2,PRODUCT_00002,5.000000
3,PRODUCT_00003,5.000000
4,PRODUCT_00004,5.000000
...,...,...
29290,PRODUCT_29466,4.035714
29291,PRODUCT_29467,5.000000
29292,PRODUCT_29468,4.600000
29293,PRODUCT_29469,4.714286


In [594]:
pr_ns3

,Product_id,Price,Counts,Total
0,PRODUCT_00000,10.91,1,10.91
1,PRODUCT_00001,248.00,1,248.00
2,PRODUCT_00002,79.80,1,79.80
3,PRODUCT_00003,37.90,1,37.90
4,PRODUCT_00004,45.87,1,45.87
...,...,...,...,...
29290,PRODUCT_29466,69.90,28,1957.20
29291,PRODUCT_29467,160.00,1,160.00
29292,PRODUCT_29468,254.50,5,636.50
29293,PRODUCT_29469,69.61,7,228.92


In [596]:
# 행 숫자 맞아서 이제 좀 다행^^

# 상관 계수 시작 ,, 

In [598]:
score_price_ns = pd.merge(rs_ns3, pr_ns3, on='Product_id', how='inner')

In [599]:
score_price_ns

,Product_id,Review_score,Price,Counts,Total
0,PRODUCT_00000,5.000000,10.91,1,10.91
1,PRODUCT_00001,5.000000,248.00,1,248.00
2,PRODUCT_00002,5.000000,79.80,1,79.80
3,PRODUCT_00003,5.000000,37.90,1,37.90
4,PRODUCT_00004,5.000000,45.87,1,45.87
...,...,...,...,...,...
29290,PRODUCT_29466,4.035714,69.90,28,1957.20
29291,PRODUCT_29467,5.000000,160.00,1,160.00
29292,PRODUCT_29468,4.600000,254.50,5,636.50
29293,PRODUCT_29469,4.714286,69.61,7,228.92


In [606]:
score_price_ns.rename(columns={"Total" : "Total_price"}, inplace=True)

In [607]:
score_price_ns

,Product_id,Review_score,Price,Counts,Total_price
0,PRODUCT_00000,5.000000,10.91,1,10.91
1,PRODUCT_00001,5.000000,248.00,1,248.00
2,PRODUCT_00002,5.000000,79.80,1,79.80
3,PRODUCT_00003,5.000000,37.90,1,37.90
4,PRODUCT_00004,5.000000,45.87,1,45.87
...,...,...,...,...,...
29290,PRODUCT_29466,4.035714,69.90,28,1957.20
29291,PRODUCT_29467,5.000000,160.00,1,160.00
29292,PRODUCT_29468,4.600000,254.50,5,636.50
29293,PRODUCT_29469,4.714286,69.61,7,228.92


In [614]:
# Seller_id 없을 때 Review_score 랑 Total_price 상관계수
review_ns_1 = score_price_ns['Review_score'].corr(score_price_ns['Total_price'])

In [615]:
review_ns_1

-0.007445772259951883

In [617]:
# Seller_id 없을 때 Review_score 랑 Counts 상관계수
review_ns_2 = score_price_ns['Review_score'].corr(score_price_ns['Counts'])

In [618]:
review_ns_2

-0.009771342784221454

# 주문 횟수 구하기

In [620]:
order_items

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69
...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72


## 셀러 있는 버전
Order_id , Product_id , Seller_id 일치하는 것들 합치고  
Order_id 제외하고 Product_id , Seller_id 일치하면 카운트 세기  
Order_item_id , Price , Freight_value 제외할 것  

In [669]:
order_counts_ys = order_items[['Order_id', 'Product_id', 'Seller_id']]

In [670]:
order_counts_ys

,Order_id,Product_id,Seller_id
0,ORDER_75477,PRODUCT_23128,SELLER_0452
1,ORDER_63604,PRODUCT_24351,SELLER_0414
2,ORDER_05566,PRODUCT_20226,SELLER_1637
3,ORDER_04632,PRODUCT_07878,SELLER_1445
4,ORDER_61452,PRODUCT_03505,SELLER_0581
...,...,...,...
100552,ORDER_70405,PRODUCT_04209,SELLER_2078
100553,ORDER_62069,PRODUCT_08229,SELLER_1593
100554,ORDER_46632,PRODUCT_21462,SELLER_0528
100555,ORDER_52966,PRODUCT_19978,SELLER_2563


order_counts_ys['Order_counts'] = order_counts_ys.groupby(['Order_id', 'Product_id', 'Seller_id'])['Order_item_id'].transform('count')

In [650]:
order_counts_ys2 = order_counts_ys.groupby(['Order_id', 'Product_id', 'Seller_id']).sum().reset_index()
# 주문 당 같은 상품/셀러인 걸 묶은 것임!
# 제품 자체의 주문 건수를 확인하고 싶어서

In [651]:
order_counts_ys2

,Order_id,Product_id,Seller_id
0,ORDER_00000,PRODUCT_02093,SELLER_0491
1,ORDER_00001,PRODUCT_26747,SELLER_2662
2,ORDER_00002,PRODUCT_10598,SELLER_1354
3,ORDER_00003,PRODUCT_16703,SELLER_1518
4,ORDER_00004,PRODUCT_08274,SELLER_1599
...,...,...,...
91293,ORDER_88083,PRODUCT_16488,SELLER_2296
91294,ORDER_88084,PRODUCT_01588,SELLER_0031
91295,ORDER_88085,PRODUCT_13335,SELLER_1892
91296,ORDER_88086,PRODUCT_07372,SELLER_2445


In [664]:
order_counts_ys2['Order_counts'] = order_counts_ys2.groupby(['Product_id', 'Seller_id'])['Order_id'].transform('count')

In [665]:
order_counts_ys2

,Order_id,Product_id,Seller_id,Order_counts
0,ORDER_00000,PRODUCT_02093,SELLER_0491,4
1,ORDER_00001,PRODUCT_26747,SELLER_2662,3
2,ORDER_00002,PRODUCT_10598,SELLER_1354,29
3,ORDER_00003,PRODUCT_16703,SELLER_1518,25
4,ORDER_00004,PRODUCT_08274,SELLER_1599,49
...,...,...,...,...
91293,ORDER_88083,PRODUCT_16488,SELLER_2296,1
91294,ORDER_88084,PRODUCT_01588,SELLER_0031,17
91295,ORDER_88085,PRODUCT_13335,SELLER_1892,3
91296,ORDER_88086,PRODUCT_07372,SELLER_2445,299


In [672]:
order_counts_ys2.loc[(order_counts_ys2['Product_id']=='PRODUCT_29466') & (order_counts_ys2['Seller_id']=='SELLER_0791')]
# 제대로 들어갔는가 확인용임!

,Order_id,Product_id,Seller_id,Order_counts
93,ORDER_00091,PRODUCT_29466,SELLER_0791,28
4062,ORDER_03926,PRODUCT_29466,SELLER_0791,28
7151,ORDER_06896,PRODUCT_29466,SELLER_0791,28
10399,ORDER_10046,PRODUCT_29466,SELLER_0791,28
10688,ORDER_10329,PRODUCT_29466,SELLER_0791,28
14147,ORDER_13691,PRODUCT_29466,SELLER_0791,28
17869,ORDER_17289,PRODUCT_29466,SELLER_0791,28
19231,ORDER_18610,PRODUCT_29466,SELLER_0791,28
24944,ORDER_24120,PRODUCT_29466,SELLER_0791,28
32877,ORDER_31769,PRODUCT_29466,SELLER_0791,28


In [671]:
order_counts_ys.loc[(order_counts_ys['Product_id']=='PRODUCT_29466') & (order_counts_ys['Seller_id']=='SELLER_0791')]

,Order_id,Product_id,Seller_id
2191,ORDER_63740,PRODUCT_29466,SELLER_0791
9507,ORDER_06896,PRODUCT_29466,SELLER_0791
10254,ORDER_77947,PRODUCT_29466,SELLER_0791
12612,ORDER_68214,PRODUCT_29466,SELLER_0791
14095,ORDER_60516,PRODUCT_29466,SELLER_0791
14196,ORDER_69857,PRODUCT_29466,SELLER_0791
21950,ORDER_86384,PRODUCT_29466,SELLER_0791
32754,ORDER_13691,PRODUCT_29466,SELLER_0791
40448,ORDER_00091,PRODUCT_29466,SELLER_0791
43843,ORDER_17289,PRODUCT_29466,SELLER_0791


In [676]:
order_counts_ys3 = order_counts_ys2.groupby(['Product_id', 'Seller_id', 'Order_counts']).sum().reset_index()

In [677]:
order_counts_ys3

,Product_id,Seller_id,Order_counts,Order_id
0,PRODUCT_00000,SELLER_0385,1,ORDER_31290
1,PRODUCT_00001,SELLER_1601,1,ORDER_03477
2,PRODUCT_00002,SELLER_0110,1,ORDER_39703
3,PRODUCT_00003,SELLER_2501,1,ORDER_52221
4,PRODUCT_00004,SELLER_1294,1,ORDER_71979
...,...,...,...,...
30653,PRODUCT_29467,SELLER_0207,1,ORDER_54807
30654,PRODUCT_29468,SELLER_0768,2,ORDER_33298ORDER_37778
30655,PRODUCT_29468,SELLER_0933,3,ORDER_00498ORDER_40405ORDER_44961
30656,PRODUCT_29469,SELLER_0578,3,ORDER_43110ORDER_46908ORDER_63094


In [678]:
order_counts_ys3.loc[(order_counts_ys3['Product_id']=='PRODUCT_29466') & (order_counts_ys3['Seller_id']=='SELLER_0791')]

,Product_id,Seller_id,Order_counts,Order_id
30652,PRODUCT_29466,SELLER_0791,28,ORDER_00091ORDER_03926ORDER_06896ORDER_10046OR...


In [682]:
order_counts_ys3.drop(columns='Order_id', inplace=True)

KeyError: "['Order_id'] not found in axis"

In [683]:
order_counts_ys3

,Product_id,Seller_id,Order_counts
0,PRODUCT_00000,SELLER_0385,1
1,PRODUCT_00001,SELLER_1601,1
2,PRODUCT_00002,SELLER_0110,1
3,PRODUCT_00003,SELLER_2501,1
4,PRODUCT_00004,SELLER_1294,1
...,...,...,...
30653,PRODUCT_29467,SELLER_0207,1
30654,PRODUCT_29468,SELLER_0768,2
30655,PRODUCT_29468,SELLER_0933,3
30656,PRODUCT_29469,SELLER_0578,3


## 구매가 2건 이상인 걸 가져와야지..

In [713]:
order_counts_ys4 = order_counts_ys3.loc[order_counts_ys3['Order_counts']>1]

In [714]:
order_counts_ys4
# 주문이 2번 이상 있었던 상품 리스트

,Product_id,Seller_id,Order_counts
5,PRODUCT_00005,SELLER_1525,2
6,PRODUCT_00006,SELLER_0791,3
9,PRODUCT_00009,SELLER_2502,2
10,PRODUCT_00010,SELLER_0585,2
15,PRODUCT_00015,SELLER_2649,2
...,...,...,...
30651,PRODUCT_29465,SELLER_0299,10
30652,PRODUCT_29466,SELLER_0791,28
30654,PRODUCT_29468,SELLER_0768,2
30655,PRODUCT_29468,SELLER_0933,3


In [722]:
rs_ys4

,Product_id,Seller_id,Review_score
0,PRODUCT_00000,SELLER_0385,5.000000
1,PRODUCT_00001,SELLER_1601,5.000000
2,PRODUCT_00002,SELLER_0110,5.000000
3,PRODUCT_00003,SELLER_2501,5.000000
4,PRODUCT_00004,SELLER_1294,5.000000
...,...,...,...
30506,PRODUCT_29467,SELLER_0207,5.000000
30507,PRODUCT_29468,SELLER_0768,4.500000
30508,PRODUCT_29468,SELLER_0933,4.666667
30509,PRODUCT_29469,SELLER_0578,4.714286


In [723]:
score_price_ys

,Product_id,Seller_id,Review_score,Counts,Total_price
0,PRODUCT_00000,SELLER_0385,5.000000,1,10.91
1,PRODUCT_00001,SELLER_1601,5.000000,1,248.00
2,PRODUCT_00002,SELLER_0110,5.000000,1,79.80
3,PRODUCT_00003,SELLER_2501,5.000000,1,37.90
4,PRODUCT_00004,SELLER_1294,5.000000,1,45.87
...,...,...,...,...,...
30506,PRODUCT_29467,SELLER_0207,5.000000,1,160.00
30507,PRODUCT_29468,SELLER_0768,4.500000,2,254.00
30508,PRODUCT_29468,SELLER_0933,4.666667,3,382.50
30509,PRODUCT_29469,SELLER_0578,4.714286,7,228.92


In [729]:
score_price_ys2 = pd.merge(order_counts_ys4, rs_ys4, on=['Product_id', 'Seller_id'], how='inner')

In [730]:
score_price_ys2

,Product_id,Seller_id,Order_counts,Review_score
0,PRODUCT_00005,SELLER_1525,2,3.500000
1,PRODUCT_00006,SELLER_0791,3,1.444444
2,PRODUCT_00009,SELLER_2502,2,5.000000
3,PRODUCT_00010,SELLER_0585,2,4.500000
4,PRODUCT_00015,SELLER_2649,2,3.500000
...,...,...,...,...
12222,PRODUCT_29465,SELLER_0299,10,4.454545
12223,PRODUCT_29466,SELLER_0791,28,4.035714
12224,PRODUCT_29468,SELLER_0768,2,4.500000
12225,PRODUCT_29468,SELLER_0933,3,4.666667


In [731]:
score_price_ys3 = pd.merge(score_price_ys2, score_price_ys, on=['Product_id', 'Seller_id', 'Review_score'], how='inner')

In [732]:
score_price_ys3

,Product_id,Seller_id,Order_counts,Review_score,Counts,Total_price
0,PRODUCT_00005,SELLER_1525,2,3.500000,2,1926.00
1,PRODUCT_00006,SELLER_0791,3,1.444444,9,89.91
2,PRODUCT_00009,SELLER_2502,2,5.000000,3,215.97
3,PRODUCT_00010,SELLER_0585,2,4.500000,2,478.00
4,PRODUCT_00015,SELLER_2649,2,3.500000,2,33.80
...,...,...,...,...,...,...
12222,PRODUCT_29465,SELLER_0299,10,4.454545,11,285.00
12223,PRODUCT_29466,SELLER_0791,28,4.035714,28,1957.20
12224,PRODUCT_29468,SELLER_0768,2,4.500000,2,254.00
12225,PRODUCT_29468,SELLER_0933,3,4.666667,3,382.50


In [748]:
review_ys_3 = score_price_ys3['Review_score'].corr(score_price_ys3['Order_counts'])

In [749]:
review_ys_3

-0.007374902367426488

In [750]:
review_ys_4 = score_price_ys3['Review_score'].corr(score_price_ys3['Counts'])

In [751]:
review_ys_4

-0.01806951197100467

In [752]:
review_ys_5 = score_price_ys3['Review_score'].corr(score_price_ys3['Total_price'])

In [753]:
review_ys_5

-0.011343359636094506

# 셀러 없는 버전

In [684]:
order_items

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69
...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72


In [685]:
order_counts_ns = order_items[['Order_id', 'Product_id']]

In [693]:
order_counts_ns

,Order_id,Product_id
0,ORDER_75477,PRODUCT_23128
1,ORDER_63604,PRODUCT_24351
2,ORDER_05566,PRODUCT_20226
3,ORDER_04632,PRODUCT_07878
4,ORDER_61452,PRODUCT_03505
...,...,...
100552,ORDER_70405,PRODUCT_04209
100553,ORDER_62069,PRODUCT_08229
100554,ORDER_46632,PRODUCT_21462
100555,ORDER_52966,PRODUCT_19978


In [695]:
order_counts_ns2 = order_counts_ns.groupby(['Order_id', 'Product_id']).sum().reset_index()

In [696]:
order_counts_ns2

,Order_id,Product_id
0,ORDER_00000,PRODUCT_02093
1,ORDER_00001,PRODUCT_26747
2,ORDER_00002,PRODUCT_10598
3,ORDER_00003,PRODUCT_16703
4,ORDER_00004,PRODUCT_08274
...,...,...
91293,ORDER_88083,PRODUCT_16488
91294,ORDER_88084,PRODUCT_01588
91295,ORDER_88085,PRODUCT_13335
91296,ORDER_88086,PRODUCT_07372


In [701]:
order_counts_ns2['Order_counts'] = order_counts_ns2.groupby(['Product_id'])['Order_id'].transform('count')

In [702]:
order_counts_ns2

,Order_id,Product_id,Order_counts
0,ORDER_00000,PRODUCT_02093,4
1,ORDER_00001,PRODUCT_26747,3
2,ORDER_00002,PRODUCT_10598,29
3,ORDER_00003,PRODUCT_16703,25
4,ORDER_00004,PRODUCT_08274,49
...,...,...,...
91293,ORDER_88083,PRODUCT_16488,1
91294,ORDER_88084,PRODUCT_01588,17
91295,ORDER_88085,PRODUCT_13335,3
91296,ORDER_88086,PRODUCT_07372,299


In [706]:
order_counts_ns3 = order_counts_ns2.groupby(['Product_id', 'Order_counts']).sum().reset_index()

In [707]:
order_counts_ns3

,Product_id,Order_counts,Order_id
0,PRODUCT_00000,1,ORDER_31290
1,PRODUCT_00001,1,ORDER_03477
2,PRODUCT_00002,1,ORDER_39703
3,PRODUCT_00003,1,ORDER_52221
4,PRODUCT_00004,1,ORDER_71979
...,...,...,...
29425,PRODUCT_29466,28,ORDER_00091ORDER_03926ORDER_06896ORDER_10046OR...
29426,PRODUCT_29467,1,ORDER_54807
29427,PRODUCT_29468,5,ORDER_00498ORDER_33298ORDER_37778ORDER_40405OR...
29428,PRODUCT_29469,3,ORDER_43110ORDER_46908ORDER_63094


In [710]:
order_counts_ns3.drop(columns='Order_id', inplace=True)

In [711]:
order_counts_ns3

,Product_id,Order_counts
0,PRODUCT_00000,1
1,PRODUCT_00001,1
2,PRODUCT_00002,1
3,PRODUCT_00003,1
4,PRODUCT_00004,1
...,...,...
29425,PRODUCT_29466,28
29426,PRODUCT_29467,1
29427,PRODUCT_29468,5
29428,PRODUCT_29469,3


In [715]:
order_counts_ns4 = order_counts_ns3.loc[order_counts_ns3['Order_counts']>1]

In [716]:
order_counts_ns4
# 주문이 2번 이상 있었던 상품 리스트

,Product_id,Order_counts
5,PRODUCT_00005,2
6,PRODUCT_00006,3
9,PRODUCT_00009,2
10,PRODUCT_00010,2
15,PRODUCT_00015,2
...,...,...
29422,PRODUCT_29463,3
29424,PRODUCT_29465,10
29425,PRODUCT_29466,28
29427,PRODUCT_29468,5


order_counts_ns4 에 평점 정보 넣어야 할 거 같은데 ?  
그리고나서 기존에 본 거랑 오더 카운츠도 하던가 해야할 듯

In [743]:
score_price_ns2 = pd.merge(order_counts_ns4, rs_ns3, on=['Product_id',], how='inner')

In [744]:
score_price_ns2

,Product_id,Order_counts,Review_score
0,PRODUCT_00005,2,3.500000
1,PRODUCT_00006,3,1.444444
2,PRODUCT_00009,2,5.000000
3,PRODUCT_00010,2,4.500000
4,PRODUCT_00015,2,3.500000
...,...,...,...
11988,PRODUCT_29463,3,4.000000
11989,PRODUCT_29465,10,4.454545
11990,PRODUCT_29466,28,4.035714
11991,PRODUCT_29468,5,4.600000


In [746]:
score_price_ns3 = pd.merge(score_price_ns2, score_price_ns, on=['Product_id', 'Review_score'], how='inner')

In [747]:
score_price_ns3

,Product_id,Order_counts,Review_score,Price,Counts,Total_price
0,PRODUCT_00005,2,3.500000,1926.00,2,1926.00
1,PRODUCT_00006,3,1.444444,9.99,9,89.91
2,PRODUCT_00009,2,5.000000,71.99,3,215.97
3,PRODUCT_00010,2,4.500000,239.00,2,478.00
4,PRODUCT_00015,2,3.500000,16.90,2,33.80
...,...,...,...,...,...,...
11988,PRODUCT_29463,3,4.000000,845.99,3,1272.98
11989,PRODUCT_29465,10,4.454545,81.00,11,285.00
11990,PRODUCT_29466,28,4.035714,69.90,28,1957.20
11991,PRODUCT_29468,5,4.600000,254.50,5,636.50


In [754]:
review_ns_3 = score_price_ns3['Review_score'].corr(score_price_ys3['Order_counts'])

In [757]:
review_ns_3

0.005139487008838128

In [755]:
review_ns_4 = score_price_ns3['Review_score'].corr(score_price_ys3['Counts'])

In [758]:
review_ns_4

0.003063220435224402

In [756]:
review_ns_5 = score_price_ns3['Review_score'].corr(score_price_ys3['Total_price'])

In [759]:
review_ns_5

0.0006488976044850778

In [761]:
order_items

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_75477,1,PRODUCT_23128,SELLER_0452,58.90,13.29
1,ORDER_63604,1,PRODUCT_24351,SELLER_0414,239.90,19.93
2,ORDER_05566,1,PRODUCT_20226,SELLER_1637,199.00,17.87
3,ORDER_04632,1,PRODUCT_07878,SELLER_1445,199.90,18.14
4,ORDER_61452,1,PRODUCT_03505,SELLER_0581,21.90,12.69
...,...,...,...,...,...,...
100552,ORDER_70405,1,PRODUCT_04209,SELLER_2078,299.99,43.41
100553,ORDER_62069,1,PRODUCT_08229,SELLER_1593,350.00,36.53
100554,ORDER_46632,1,PRODUCT_21462,SELLER_0528,99.90,16.95
100555,ORDER_52966,1,PRODUCT_19978,SELLER_2563,55.99,8.72


In [768]:
order_items.loc[order_items['Product_id']=='PRODUCT_29466']

,Order_id,Order_item_id,Product_id,Seller_id,Price,Freight_value
2191,ORDER_63740,1,PRODUCT_29466,SELLER_0791,69.9,35.34
9507,ORDER_06896,1,PRODUCT_29466,SELLER_0791,69.9,28.80
10254,ORDER_77947,1,PRODUCT_29466,SELLER_0791,69.9,28.80
12612,ORDER_68214,1,PRODUCT_29466,SELLER_0791,69.9,15.35
14095,ORDER_60516,1,PRODUCT_29466,SELLER_0791,69.9,26.11
14196,ORDER_69857,1,PRODUCT_29466,SELLER_0791,69.9,28.80
21950,ORDER_86384,1,PRODUCT_29466,SELLER_0791,69.9,26.11
32754,ORDER_13691,1,PRODUCT_29466,SELLER_0791,69.9,14.60
40448,ORDER_00091,1,PRODUCT_29466,SELLER_0791,69.9,26.11
43843,ORDER_17289,1,PRODUCT_29466,SELLER_0791,69.9,26.11
